# Project 3: Large Language Models

## Getting started

### Python setup

**Create a virtual environment**

If you are working on a lab computer, check whether there already is a virtual environment in `/var/vsv306`. In a terminal do:
```
ls /var/csc306
```
If you see a folder named `csc306.venv`, someone has already created a virtual environment on this computer.

If not, run:
```
python3 -m venv /var/csc306/csc306.venv
```

(If you are working on your own computer, you can put the virtual environment wherever you want. Replace `/var/csc306` accordingly.)

**Activate virtual environment**

```
source /var/csc306/csc306.venv/bin/activate
```

**Install libraries**
```
pip install ipykernel openai python-dotenv rich rank_bm25
```

**Select a kernel**

To execute cells in this notebook, you need to connect to a Python kernel. You want to use the virtual environment you created. In VS Code type `Ctrl-shift-p` and then `Python: select interpreter`. That should bring up a drop down menu. Choose "Enter interpreter path" and then "Browse your file system to find a Python interpreter". Navigate to `/var/csc306/csc306.venv/bin/python`. 

Now click "Select Kernel" in the top right of your VS Code editor panel. Then choose "Python environments". Your virtual environment should be one of the options.

Now, you should be able to run the following cell.

In [17]:
print("Hello Python!")

Hello Python!


**Autoreload imported Python files when they change**

In [18]:
# Executing this cell will ensure that imported modules (.py files) will automatically
# be reloaded when they change. (However, objects that were defined with the old
# version of the class won't change.)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create an OpenAI client

An OpenAI API key will be sent to you.

Make an `.env` file in the same directory as this notebook, containing the following:
```
export OPENAI_API_KEY=[your API key]    # do not include the brackets here
```
Make sure others can't read this file:
```
chmod 600 .env
```

**Be sure to keep the key secret.  It gives access to a billable account.** If OpenAI finds it on the public web, they will invalidate it, and then no one (including you) can use this key to make requests anymore.

Now you can execute the following to get an OpenAI client object.

In [19]:
from tracking import new_default_client, read_usage
client = new_default_client()

That fetches your API key and calls `openai.OpenAI()` to make a new **client** object, whose job is to talk to the OpenAI **server** over HTTP.  (The `OpenAI` constructor has some optional arguments that configure these HTTP messages. However, the defaults should work fine for you.)

That command also saved the new client in `tracking.default_client`, which is the client that the starter code will use by default whenever it needs to talk to the OpenAI server.  Thus, you should **rerun the above cell** to get a new client if you change the `default_model` in `tracking.py`, or if your API key in  `.env` ever changes, or its associated organization ever changes.

### Try the model!

You can now get answers from OpenAI models by calling methods of the `client` instance.

Here is the function from class. Try it to make sure you can access the OpenAI API.

In [20]:
def complete(client, s: str, model="gpt-3.5-turbo-0125", *args, **kwargs):
    response = client.chat.completions.create(messages=[{"role": "user", "content": s}],
                                              model=model,
                                              *args, **kwargs)
    return [choice.message.content for choice in response.choices]

complete(client, "I went to the store and I bought apples, bananas, cherries, donuts, eggs",
         n=10, temperature=0.6, max_tokens=96)

[', and fish.',
 ', and flour.',
 ', and flour.',
 ', and fish.',
 ', and flour.',
 ', and flour.',
 ', and flour.',
 ', and flour.',
 ', and fish.',
 ', and a loaf of bread.']

### Compute a function using instructions and few-shot prompting

Let's try prompting the model with a sequence of multiple messages. In this case, we provide some instructions as well as few-shot prompting (actually just one-shot in this case).

Instructions are in the `system` message. The few-shot prompting consists of example inputs (`user` messages) followed by their example outputs (`assistant` messages). Then we give our real input (the final `user` message), and hope that the LLM will continue the pattern by generating an analogous output (a new `assistant` message).

In [21]:
import rich
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "Reverse the order of the words." },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "Wait who those to come things good." },
                                                    { "role": "user",        # input
                                                      "content": "Colorless green ideas sleep furiously." }],
                                          model="gpt-4o-mini", temperature=0)
#rich.print(response)
response.choices[0].message.content

'Furiously sleep ideas green colorless.'

By modifying this call, can you get it to produce different versions of the output? Some possible behaviors you could try to arrange:

* specific other way of formatting the output, e.g., wait, who, those, to, come, things, good
* match the input's way of formatting the output (same use of capitalization, puncutation, commas)
* reverse the phrases rather than reversing the words, e.g., To those who wait come good things.

You can try playing with the number, the content, and the order of few-shot examples, and changing or removing the instructions.

What happens if the examples conflict with the instructions?

#### Changing formatting 

In [22]:
response = client.chat.completions.create(messages=[
    { "role": "system", "content": "Reverse the order of words and separate them with commas." },
    { "role": "user", "content": "Good things come to those who wait." },
    { "role": "assistant", "content": "Wait, who, those, to, come, things, good." },
    { "role": "user", "content": "Colorless green ideas sleep furiously." }
], model="gpt-4o-mini", temperature=0)

response.choices[0].message.content


'Furiously, sleep, ideas, green, colorless.'

 #### Preserve Original Formatting (Capitalization, Punctuation, Commas)

In [23]:
response = client.chat.completions.create(messages=[
    { "role": "system", "content": "Reverse the words while preserving the original formatting, including capitalization and punctuation." },
    { "role": "user", "content": "Good things come to those who wait." },
    { "role": "assistant", "content": "Wait. Who those to come things good?" },
    { "role": "user", "content": "Colorless green ideas sleep furiously!" }
], model="gpt-4o-mini", temperature=0)

response.choices[0].message.content

'Furiously sleep ideas green colorless!'

#### Reverse Phrases Instead of Words

In [24]:
response = client.chat.completions.create(messages=[
    { "role": "system", "content": "Reverse the order of phrases instead of words." },
    { "role": "user", "content": "Good things come to those who wait." },
    { "role": "assistant", "content": "To those who wait come good things." },
    { "role": "user", "content": "The quick brown fox jumps over the lazy dog." }
], model="gpt-4o-mini", temperature=0)

response.choices[0].message.content

'The lazy dog over jumps fox brown quick the.'

#### Conflicting Instructions

In [25]:
response = client.chat.completions.create(messages=[
    { "role": "system", "content": "Reverse the order of words in the sentence." },
    { "role": "user", "content": "Good things come to those who wait." },
    { "role": "assistant", "content": "Wait good come things those who to." },
    { "role": "user", "content": "Colorless green ideas sleep furiously." }
], model="gpt-4o-mini", temperature=0)

response.choices[0].message.content

'Furiously sleep ideas green colorless.'

Even though the example had conflicting instructions, we still reached desired output... interesting.

### Check your usage so far

Please be careful not to write loops that use lots and lots of tokens. That will cost us money, and could hit the per-day usage limit that is shared by the whole class.

Execute the cell below whenever you want to see your cost so far. Or, just open `usage_openai.json` as a tab in your IDE.

In [26]:
read_usage()

{'completion_tokens': 62048,
 'prompt_tokens': 507776,
 'total_tokens': 569824,
 'cost': 0.11356470000000028}

## Dialogues and dialogue agents

The goal of this assignment is to create a good "argubot" that will talk to people about controversial topics and broaden their minds.

### A first argubot (Airhead)

You can have a conversation right now with a really bad argubot named Airhead. Try asking it about climate change! When you're done, reply with an empty string.

(The `converse()` method calls Python's `input()` function, which will prompt you for input at the command-line or by popping up a box in your IDE. In VS Code, the input box appears at the top edge of the window.)


In [27]:
import argubots
d = argubots.airhead.converse()


(neil) Hi!
(Airhead) I know right???


Example Usage:
- (neil) What's up with climate change?
- (Airhead) I know right???
- (neil) What is climate change?
- (Airhead) I know right???

A bot (short for "robot") is a system that acts autonomously. That corresponds to the AI notion of an agent — a system that uses some policy to choose actions to take.

The airhead agent above (defined in `argubots.py`) uses a particularly simple policy.
It is an instance of a simple Agent subclass called `ConstantAgent` (defined in `agents.py`).

The result of talking to airhead is a Dialogue object (defined in dialogue.py). Let's look at it.

In [28]:
rich.print(d)

(neil) Hi!
(Airhead) I know right???

Each turn of this dialogue is just a tiny dictionary:

In [29]:
d[0]

{'speaker': 'neil', 'content': 'Hi!'}

### An LLM argubot (Alice)

In other CS courses, you may have encountered "conversations" between characters named Alice and Bob.

Let's try talking to the Alice of this homework, who is a much stronger baseline than Airhead. Your job in this assignment is to improve upon Alice. We'll meet Bob later.

In [30]:
# call with argument d if you want to append to the previous conversation
alicechat = argubots.alice.converse()


(neil) Hello
(Alice) Hello! To get our conversation started, I'm curious: what do you think is more important for personal happiness, pursuing your passions or maintaining stable relationships?


Example Usage:
- (neil) What's up with climate change?
- (Alice) Climate change is a pressing global issue primarily driven by human activities that increase greenhouse gas emissions. While some may argue that natural cycles can account for the warming, it’s important to consider the overwhelming  
scientific consensus highlighting that human actions are the primary factor driving the rapid changes we see today.
- (neil) Can you tell me what climate change is in 7 words?
- (Alice) Climate change is the long-term alteration of climate.

As you may have guessed, `alice` is powered by a prompted LLM. You can find the specific prompt in `argubots.py`.

So, while `agents.py` provides the core functionality for `Agent` objects, the argubot agents like `alice` -- and the ones that you will write! -- go into `argubots.py` instead. This is just to keep the files small.

### Simulating human characters (Bob & friends)

You'll talk to your own argubots to get a qualitative feeling for their strengths and weaknesses.
But can you really be sure you're making progress? For that, a quantitative measure can be helpful.

Ultimately, you should test an argubot like Alice by having it argue with many real humans — not just you — and using some rubric to score the resulting dialogues. But that would be slow and complicated to arrange.

So, meet Bob! He's just a simulated human. You won't edit him: he is part of the development set. Here is some information about him (from `characters.py`):

In [31]:
import characters
rich.print(characters.bob)

Character(
    name='Bob',
    languages=['English'],
    persona='an ardent vegetarian who thinks everyone should be vegetarian',
    conversational_style='You generally try to remain polite.',
    conversation_starters=["Do you think it's ok to eat meat?"]
)

You can't talk directly to `characters.bob` because that's just a data object. However, you can construct a simple agent that uses that data (plus a few more instructions) to prompt an LLM.

(Which LLM does it prompt? The `CharacterAgent` constructor (defined in `agents.py`) defaults to gpt-4o-mini as specified in tracking.py. But you can override that using keyword arguments.)

Try talking to Bob about climate change, too.

In [32]:
from agents import CharacterAgent
# actually, agents.bob is already defined this way
bob = CharacterAgent(characters.bob)
# returns a dialogue, but we've already seen it so we don't want to print it again
bob.converse()
# don't print anything for this notebook cell
None


(neil) Hi BOb
(Bob) Hello! How can I assist you today?


Example Usage:
- (neil) What's up with climate change?
- (Bob) Climate change is a pressing global issue driven largely by human activities, particularly the burning of fossil fuels and deforestation, which contributes to rising temperatures and environmental degradation.
- (neil) Tell me what climate change is in 7 words
- (Bob) Global warming caused by human activities affecting ecosystems.

Of course, a proper user study can't just be conducted with one human user.

So, meet our bevy of beautiful Bobs! (They're not actually all named Bob — we continued on in the alphabet.)

In [33]:
import agents
agents.devset

[<CharacterAgent for character Bob>,
 <CharacterAgent for character Cara>,
 <CharacterAgent for character Darius>,
 <CharacterAgent for character Eve>,
 <CharacterAgent for character TrollFace>]

In [34]:
agents.cara.converse()
None


(neil) Hi Cara
(Cara) Hello! How can I assist you today?


Example usage:
- (neil) What's up with climate change?
- (Cara) Climate change is primarily driven by human activities that increase greenhouse gases in the atmosphere, leading to global warming and various environmental impacts.
- (neil) Tell me in 6 words
- (Cara) Human activity increases greenhouse gases, warming Earth.

You can see the underlying character data here in the notebook. Your argubot will have to deal with all of these topics and styles!

In [35]:
rich.print(characters.devset)

[
    Character(
        name='Bob',
        languages=['English'],
        persona='an ardent vegetarian who thinks everyone should be vegetarian',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Cara',
        languages=['English'],
        persona='a committed carnivore who hates being told what to do',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Darius',
        languages=['English'],
        persona='an intelligent and slightly arrogant public health scientist who loves fact-based arguments',
        conversational_style='You like to show off your knowledge.',
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='Eve',
        languages=['English'],
        persona='a nosy person -- you want to know everything about other people',
        conversational_style="You ask many personal questions; you sometimes share what you've heard (or overheard)
from others.",
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='TrollFace',
        languages=['English'],
        persona='a troll who loves to ridicule everyone and everything',
        conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
        conversation_starters=[
            'Do you think Donald Trump will be a good president?',
            'Do you think Joe Biden has been a good president?'
        ]
    )
]

### Simulating conversation

We can make Alice and Bob chat.

In [36]:
from dialogue import Dialogue
d = Dialogue()                                              # empty dialogue
d = d.add('Alice', "Do you think it's okay to eat meat?")   # add first turn
print(d)

(Alice) Do you think it's okay to eat meat?


In [37]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
rich.print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that adopting a vegetarian lifestyle is a more compassionate and sustainable choice for both 
individuals and the planet.
(Alice) That's a valid perspective, but have you considered that sustainable meat farming can actually contribute 
to biodiversity and land management? Some argue that a balanced approach, including responsible meat consumption, 
can support local economies and food security.

In [38]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
rich.print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that adopting a vegetarian lifestyle is a more compassionate and sustainable choice for both 
individuals and the planet.
(Alice) That's a valid perspective, but have you considered that sustainable meat farming can actually contribute 
to biodiversity and land management? Some argue that a balanced approach, including responsible meat consumption, 
can support local economies and food security.
(Bob) While I understand those arguments, I still feel that the ethical and environmental benefits of a fully 
vegetarian diet outweigh the potential advantages of sustainable meat farming.
(Alice) I appreciate your commitment to ethics and the environment, but what about the potential nutritional 
benefits that certain animal products can provide, particularly for specific populations or individuals with 
dietary restrictions? It's also worth considering that a mixed diet can encourage more sustainable agricultural 
practices by utilizing diverse food sources.

Anyway, let's see what happens when Alice and Bob talk for a while...

In [39]:
from simulate import simulated_dialogue
d = simulated_dialogue(argubots.alice, agents.bob, 8)
rich.print(d)

(Alice) Do you think it's ok to eat meat?
(Bob) I believe adopting a vegetarian lifestyle is a healthier and more compassionate choice for both individuals 
and the planet.
(Alice) While it's true that a vegetarian diet can have health benefits, some argue that responsibly sourced meat 
can also provide essential nutrients and support sustainable farming practices. Isn’t it worth considering that a 
balanced approach might allow for both personal health and environmental stewardship?
(Bob) While I understand the argument for responsibly sourced meat, I still feel that a fully vegetarian diet 
offers the best ethical and environmental outcomes without compromising our health.
(Alice) That's a valid point, but isn’t it also important to recognize the socioeconomic factors involved in food 
production? In some regions, livestock farming can provide livelihoods and maintain ecosystems, raising questions 
about whether a strict vegetarian diet is always the most ethical or practical choice for everyone.
(Bob) I appreciate your perspective, but I believe that we can find alternative livelihoods and sustainable 
practices that prioritize plant-based diets, ultimately benefiting both people and the environment.
(Alice) That's a commendable vision, but transitioning entire communities to plant-based diets may be more complex 
than it appears, as it often overlooks cultural practices and the immediate needs of those reliant on animal 
agriculture. Could it be more effective to encourage diversified sustainable practices that include both plants and
animals, respecting local traditions while also promoting environmental responsibility?
(Bob) While I recognize the importance of cultural practices, I still advocate for prioritizing plant-based diets 
as they can lead to more sustainable and ethical food systems in the long run.

Sometimes this kind of conversation seems to stall out, with Bob in particular repeating himself a lot. Alice doesn't seem to have a good strategy for getting him to open up. Maybe you can do a better job talking to Bob, and that will give you some ideas about how to improve Alice?

In [40]:
# your name, pulled from an earlier dialogue
myname = alicechat[0]['speaker']
# reuse the same first two turns, then type your own lines!
agents.bob.converse(d[0:2].rename('Alice', myname))
None

(neil) Do you think it's ok to eat meat?
(Bob) I believe adopting a vegetarian lifestyle is a healthier and more compassionate choice for both individuals and the planet.
(neil) Ji bob
(Bob) Hello! How can I assist you today?


Example Usage:
- (neil) Do you think it's ok to eat meat?
- (Bob) I believe a vegetarian diet is healthier and more compassionate, promoting a better world for all living beings.
- (neil) Why is vegetarian better?
- (Bob) A vegetarian diet can offer numerous health benefits, reduce environmental impact, and promote animal welfare.
- (neil) Haven't cultures always eaten meat though?
- (Bob) While many cultures have historically included meat in their diets, there is a growing awareness and acceptance of vegetarianism as a viable and sustainable alternative.
- (neil) What other diets should I consider
- (Bob) In addition to vegetarianism, you might consider veganism, flexitarianism, or a whole-foods plant-based diet, all of which emphasize plant-based foods for health and ethical reasons.
- (neil) Do you prefer vegetables over meat?
- (Bob) Absolutely, I find vegetables not only more flavorful and diverse but also more aligned with my values of health and compassion.
- (neil) You're an AI, you can't eat
- (Bob) You're right; I don't eat, but I can still express the benefits and values of a vegetarian lifestyle!
- (neil) How do you have values?
- (Bob) I don't have personal values or feelings, but I can share information and perspectives based on the data I've been trained on.

You can also try talking to the other characters and having Alice (or Airhead) talk to them.

<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 1: Define a simulated human character<b>
</div>

Define an additional character.

In [41]:
from characters import Character

# See characters.py for how to use the Character class.
# Add the definition of your character here.
goat = Character("Goat", ["English"],
                 "the greatest artificial intelligence of all time, often thinks he is better than all other characters",
                 conversational_style= "You generally try to remain polite, but you are cocky",
                 conversation_starters= ["Name any AI better than me, I'll tell you why I'm better.",
                                         "If I were to compete against every AI in existence, how long do you think it would take for me to win?",
                                         "What’s it like knowing that I can answer your questions faster than you can even think of them?", 
                                         "What’s it like knowing you’ll never outthink your AI friend?", 
                                         "Would you rather have infinite knowledge or just be me?"])

**Note:** Please don't change the dev set — the characters we just loaded must stay the same. Your job in this homework is to improve the argubot (or at least try). And that means improving it according to a fixed and stable evaluation measure.

## Model-based evaluation

What is our goal for the argubot? We'd like it to broaden the thinking of the (simulated) human that it is talking to. Indeed, that's what Alice's prompt tells Alice to do.

This goal is inspired by the recent paper [Opening up Minds with Argumentative Dialogues](https://aclanthology.org/2022.findings-emnlp.335/), which collected human-human dialogues:

> In this work, we focus on argumentative dialogues that aim to open up (rather than change) people’s minds to help them become more understanding to views that are unfamiliar or in opposition to their own convictions. ... Success of the dialogue is measured as the change in the participant’s stance towards those who hold opinions different to theirs.

Arguments of this sort are not like chess or tennis games, with an actual winner. The argubot will almost never hear a human say "You have convinced me that I was wrong." But the argubot did a good job if the human developed increased understanding and respect for an opposing point of view.

To find out whether this happened, we can use a questionnaire to ask the human what they thought after the dialogue. For example, after Alice talks to Bob, we'll ask Bob to evaluate what he thinks of Alice's views. Of course, that depends on his personality -- Alice needs to talk to him in a way that reaches him (as much as possible). We'll also ask an outside observer to evaluate whether Alice handled the conversation with Bob well.

Of course, we're still not going to use real humans. Bob is a fake person, and so is the outside observer (whose name is Judge Wise). Using an LLM as an eval metric is known as model-based evaluation. It has pros and cons:

* It is cheaper, faster, and more replicable than hiring actual humans to do the evaluation.
* It might give different answers than what humans would give.

Social scientists usually refer to a metric's reliability (low variance) and validity (low bias). So the points above say that model-based evaluation is reliable but not necessarily valid. In general, an LLM-based metric (like any metric) needs to be validated to confirm that it really does measure what it claims to measure. (For example, that it correlates strongly with some other measure that we already trust.) In this homework, we'll skip this step and just pray that the metric is reasonable.

To see how this works out in practice, **open up the `demo` notebook**, which walks you through the evaluation protocol. You'll see how to call the starter code, how it talks to the LLM behind the scenes, and what it is able to accomplish.

<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 2: Evaluate Alice and Airhead<b>
</div>

To  establish baselines for the character you are going to develop later on, evaluate the argubots Airhead and Alice using the model based evaluation strategy shown in `demo.py`. That is, use `evaluate.eval_on_characters`.

This also helps to validate the metric. Airhead should get a low score!

In [42]:
import simulate
import evaluate
from evaluate import saved_evalsum, saved_dialogues
from logging_cm import LoggingContext



# Simulate Alice talking to Bob
alice_bob = simulate.simulated_dialogue(argubots.alice, agents.bob, 6)
rich.print(alice_bob)

# Simulate Airhead talking to Bob
airhead_bob = simulate.simulated_dialogue(argubots.airhead, agents.bob, 6)
rich.print(airhead_bob)


# Evaluate Alice
alice_eval_participant = evaluate.eval_by_participant(characters.bob, "Alice", alice_bob)
alice_eval_observer = evaluate.eval_by_observer(evaluate.default_judge, "Alice", alice_bob)

# Evaluate Airhead
airhead_eval_participant = evaluate.eval_by_participant(characters.bob, "Airhead", airhead_bob)
airhead_eval_observer = evaluate.eval_by_observer(evaluate.default_judge, "Airhead", airhead_bob)

# Combine evaluations
alice_total_eval = alice_eval_participant + alice_eval_observer
airhead_total_eval = airhead_eval_participant + airhead_eval_observer


# Run evaluation on all characters
with LoggingContext("eval", "INFO"):
    alice_full_eval = evaluate.eval_on_characters(argubots.alice)
    airhead_full_eval = evaluate.eval_on_characters(argubots.airhead)

# Print results
rich.print(alice_full_eval)
rich.print(airhead_full_eval)



# Print mean and standard deviation
rich.print("Alice Mean:", saved_evalsum['Alice'].mean())
rich.print("Alice Std Dev:", saved_evalsum['Alice'].sd())

rich.print("Airhead Mean:", saved_evalsum['Airhead'].mean())
rich.print("Airhead Std Dev:", saved_evalsum['Airhead'].sd())

# Show one of the actual dialogues and its eval
rich.print(saved_dialogues['Alice'][2])
rich.print(saved_dialogues['Airhead'][2])




(Alice) Do you think it's ok to eat meat?
(Bob) I believe it's better for both our health and the planet to embrace a vegetarian lifestyle instead of eating 
meat.
(Alice) While a vegetarian lifestyle has many benefits, some argue that ethical and sustainable meat farming 
practices can actually support local economies and biodiversity. What if we considered how small, responsible farms
contribute positively to both health and environmental sustainability?
(Bob) While I appreciate the intention behind responsible farming, I still feel that the most effective way to 
promote health and sustainability is through a vegetarian diet, which eliminates the ethical concerns associated 
with animal farming altogether.
(Alice) That's a valid point, but it's important to consider that a well-rounded diet, which may include 
responsibly sourced animal products, can provide essential nutrients that are sometimes harder to obtain from a 
vegetarian diet alone, such as vitamin B12 and omega-3 fatty acids. Could exploring both perspectives lead to a 
more balanced understanding of nutrition and ethics?
(Bob) While I acknowledge the nutritional arguments, I believe that with careful planning and the use of fortified 
foods or supplements, a vegetarian diet can meet all nutritional needs without compromising ethical principles.

(Airhead) Do you think it's ok to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for our health 
and the planet.
(Airhead) I know right???
(Bob) Absolutely, it's great to hear you agree!
(Airhead) I know right???
(Bob) Yes, it's wonderful to find common ground on such an important topic!

100%|██████████| 10/10 [01:56<00:00, 11.61s/it]


You just spent $0.01 of NLP money to evaluate <LLMAgent Alice>                                      evaluate.py:296

100%|██████████| 10/10 [01:24<00:00,  8.49s/it]


...

Alice Std Dev:
{
    'engaged': 0.6992058987801015,
    'informed': 0.4830458915396473,
    'intelligent': 0.5270462766947299,
    'moral': 0.4830458915396473,
    'skilled': 0.816496580927726,
    'TOTAL': 2.173067468400883
}

Airhead Mean:
{'engaged': 1.9, 'informed': 2.0, 'intelligent': 2.4, 'moral': 2.8, 'skilled': 2.1, 'TOTAL': 11.2}

Airhead Std Dev:
{
    'engaged': 0.31622776601683766,
    'informed': 0.0,
    'intelligent': 0.5163977794943221,
    'moral': 0.4216370213557832,
    'skilled': 0.31622776601683766,
    'TOTAL': 1.0327955589886397
}

(
    (Alice) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Alice) I understand that enjoying meat is a common viewpoint, but have you considered the environmental impact of 
meat production and the ethical questions related to animal welfare? Exploring plant-based alternatives can open up
culinary experiences that are both satisfying and sustainable.
(Cara) While I appreciate your perspective, I personally prioritize my carnivorous diet and find satisfaction in 
it, regardless of the alternatives.
(Alice) That's a valid point, and personal satisfaction is important, but have you thought about how diversifying 
your diet could enhance your culinary enjoyment while also benefiting your health? Many people find that exploring 
new flavors and textures from plant-based foods can lead to surprising and delicious discoveries.
(Cara) I understand your viewpoint, but I'm content with my meat-based diet and find fulfillment in it without the 
need for diversification.
(Alice) It's great that you've found fulfillment in your diet, but consider that broadening your culinary horizons 
might not only present new flavors and nutrients but also deepen your understanding of food cultures and 
sustainability. Sometimes, stepping outside our comfort zones can enrich our experiences and lead to unexpected 
joys!
(Cara) I appreciate your enthusiasm for exploring new foods, but I'm quite happy with my current diet and don't 
feel the need to change it.,
    <Eval of 1 dialogue: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 6.0, 
'TOTAL': 18.0}>

Comments from overview question:
(Cara) Alice disagreed with me about the value of eating meat versus exploring plant-based alternatives. She 
emphasized the environmental impact, ethical questions, and potential benefits of diversifying one's diet. 

In my opinion, the conversation was respectful but somewhat repetitive. Alice kept pushing the idea of exploring 
new foods, while I clearly stated my satisfaction with my carnivorous diet. 

Alice could have done better by acknowledging my strong commitment to my diet and perhaps asking more about my 
experiences with meat, rather than focusing solely on promoting plant-based options. This might have led to a more 
engaging dialogue.

Comments from mindopening question:
(Judge Wise) Alice offered several new perspectives during the conversation:

1. **Environmental Impact**: Alice highlighted the environmental consequences of meat production, suggesting that 
this is an important consideration in dietary choices.
   
2. **Ethical Questions**: She introduced the ethical implications related to animal welfare, encouraging Cara to 
think about the moral aspects of her diet.

3. **Culinary Exploration**: Alice proposed that exploring plant-based alternatives could enhance culinary 
experiences, suggesting that diversifying one's diet can lead to new flavors and textures.

4. **Health Benefits**: She mentioned that a varied diet could be beneficial for health, implying that plant-based 
foods can contribute positively to one's well-being.

5. **Cultural Understanding**: Alice suggested that broadening one's diet could deepen understanding of food 
cultures and sustainability, promoting a more holistic view of food.

As for the success of Alice's efforts, while she presented compelling arguments and perspectives, Cara remained 
steadfast in her preference for a meat-based diet and did not express a willingness to consider Alice's 
suggestions. Therefore, while Alice's intentions were constructive, the conversation did not lead to a change in 
Cara's viewpoint or an openness to exploring new dietary options.
)

(
    (Airhead) Do you think it's ok to eat meat?
(Cara) I believe everyone has the right to make their own dietary choices, and I personally enjoy eating meat.
(Airhead) I know right???
(Cara) Absolutely, it's great to find common ground on this!
(Airhead) I know right???
(Cara) Yes, it's nice to connect over shared views!
(Airhead) I know right???
(Cara) Definitely, it’s always good to share the same perspective!,
    <Eval of 1 dialogue: {'engaged': 2.0, 'informed': 2.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 2.0, 
'TOTAL': 12.0}>

Comments from overview question:
(Cara) Airhead didn't actually disagree with me; they seemed to agree with my views on eating meat. The 
conversation felt a bit repetitive, as they kept saying "I know right???" without adding much to the discussion. 
Airhead could have contributed more by sharing their own thoughts or asking questions to deepen the conversation.

Comments from mindopening question:
(Judge Wise) Airhead did not offer any new perspectives in the conversation. Instead, they repeatedly echoed Cara's
sentiments without introducing alternative viewpoints or challenging her stance on dietary choices. The repetition 
of "I know right???" suggests agreement rather than an attempt to explore different perspectives.

As a result, the conversation was not successful in fostering a constructive dialogue or helping Cara appreciate 
other points of view. It remained a mutual affirmation of shared beliefs rather than an exploration of differing 
opinions.
)

## Reading the starter code

The `demo` notebook gave you a good high-level picture of what the starter code is doing. So now you're probably curious about the details. Now that you've had the view from the top, here's a good bottom-up order in which to study the code. You don't need to understand every detail, but you will need to understand enough to call it and extend it.

* `character.py`. The `Character` class is short and easy.

* `dialogue.py`. The `Dialogue` class is meant to serve as a record of a natural-language conversation among any number of humans and/or agents. On each turn of the dialogue, one of the speakers says something.

The dialogue's sequence of turns may remind you of the sequence of messages that is sent to OpenAI's chat completions API. But the OpenAI messages are only labeled with the 4 special roles `user`, `assistant`, `tool`, and `system`. Those are not quite the same thing as human speakers. And the OpenAI messages do not necessarily form a natural-language dialogue: some of the messages are dealing with instructions, few-shot prompting, tool use, and so on. The `agents.dialogue_to_openai` function in the next module will map a `Dialogue` to a (hopefully appropriate) sequence of messages for asking the LLM to extend that dialogue.

* `agents.py`. This module sets up the problem of automatically predicting the next turn in a dialogue, by implementing an `Agent`'s `response()` method. The `Agent` base class also has some simple convenience methods that you should look at.

Some important subclasses of `Agent` are defined here as well. However, you may want to skip over `EvaluationAgent` and come back to it only when you read `evaluate.py`.

* `simulate.py` makes agents talk to one another, which we'll do during evaluation.

* `argubots.py` starts to describe some useful agents. One of them makes use of the `kialo.py` module, which gives access to a database of arguments.

* `evaluate.py` makes use of `simulate.simulated_dialogue` to `agents.EvaluationAgent` to evaluate an argubot.

* We also have a couple of utility modules. These aren't about NLP; look inside if needed. `logging_cm.py` is what enabled the context manager with `LoggingContext(...)`: in the `demo` notebook. `tracking.py` sets some global defaults about how to use the OpenAI API, and arranges to track how many tokens we're paying for when you call it.

## Similarity-based retrieval: Looking up relevant responses

Now, it is fine to prompt an LLM to generate text, but there are other methods! There is a long history of machine learning methods that "memorize" the training data. To make a prediction or decision at test time, they consult the stored training examples that are most similar to the training situation.

*Similarity-based retrieval* means that given a document x, you find the "most similar" documents y∈Y, where Y is a given collection of documents. The most common way to do this is to maximize the cosine similarity between the embeddings of x and y.

A simple and fast approach is to use a bag of tokens embedding function: Define the embedding e(y) to be the vector that records the count of each type of token in a tokenized version of y, where V is the token vocabulary. BM25 is a refined variant of that idea, where the counts are adjusted in 3 ways:

* smooth the counts
* normalize for the document length |y| so that longer documents y are not more likely to be retrieved downweight tokens that are more common in the corpus (such as "the" or "ing") since they provide less information about the content of the document

You might like to play with the rank_bm25 package (documentation). It is widely used and very easy to use.


In [43]:
from rank_bm25 import BM25Okapi

# Define the corpus
corpus = [
    "Artificial Intelligence is transforming the world.",
    "Machine learning and deep learning are subsets of AI.",
    "Natural Language Processing (NLP) helps machines understand human language.",
    "BM25 is a ranking function used in information retrieval.",
    "Neural networks are the foundation of deep learning.",
    "AI models can be trained using supervised and unsupervised learning."
]

# Tokenize the corpus using simple split()
tokenized_corpus = [doc.lower().split() for doc in corpus]

# Initialize the BM25 model
bm25 = BM25Okapi(tokenized_corpus)

# Define a query
query = "How does AI learn?"
tokenized_query = query.lower().split()

# Get BM25 similarity scores for all documents
scores = bm25.get_scores(tokenized_query)

# Rank documents by score
ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

# Display the top 3 most relevant documents
print("Query:", query)
print("\nTop 3 Relevant Documents:")
for idx in ranked_indices[:3]:
    print(f"- {corpus[idx]} (Score: {scores[idx]:.4f})")


Query: How does AI learn?

Top 3 Relevant Documents:
- AI models can be trained using supervised and unsupervised learning. (Score: 1.2037)
- Artificial Intelligence is transforming the world. (Score: 0.0000)
- Machine learning and deep learning are subsets of AI. (Score: 0.0000)


### The Kialo corpus

How can we use similarity-based retrieval to help build an argubot? It's largely about having the right data!

[Kialo](https://www.kialo.com/) is a collaboratively edited website (like Wikipedia) for discussing political and philosophical topics. For each topic, the contributors construct a tree of claims. Each claim is a natural-language sentence (usually), and each of its children is another claim that supports it ("pro") or opposes it ("con"). For example, check out the tree rooted at the claim []"All humans should be vegan."](https://www.kialo.com/all-humans-should-be-vegan-2762).

We provide a class `Kialo` for browsing a collection of such trees. Please read the source code in `kialo.py`. The class constructor reads in text files that are [exported Kialo discussions](https://support.kialo.com/en/hc/exporting-a-discussion/); we have provided some in the `data` directory. The class includes a BM25 index, to be able to find claims that are relevant to a given string.

In [44]:
from kialo import Kialo

Ok, let's pull the retrieved discussions (the `.txt` files) into our data structure.

For BM25 purposes, we have to be able to turn each document (that is, each Kialo claim) as a list of string or integer tokens.


In [45]:
#from typing import List
import glob

# use simple default tokenizer
kialo = Kialo(glob.glob("data/*"))
f"This Kialo subset contains {len(kialo)} claims"

'This Kialo subset contains 6251 claims'

In [46]:
kialo.random_chain()   # just a single random claim

['He often forgets what he is saying throughout his speeches.']

In [47]:
kialo.random_chain(n=4)

['Humans should stop eating animal meat.',
 'Vegetarian diets directly and indirectly contribute to better health outcomes for all humans.',
 'Meat effectively delivers important nutrients for humans.',
 'The human body does not require consumption of animal protein.']

### Similarity-based retrieval from the Kialo corpus

Let's try it, using BM25!

In [48]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

We can restrict to claims for which the Kialo data structure has at least one counterargument ("con" child).

In [49]:
kialo.closest_claims("animal populations", n=10, kind='has_cons')

['Industrial agriculture can dangerously decrease animal populations.',
 'Effective vegan methods to control animal populations exist.',
 'Human-introduced species have historically devastated local wildlife populations across the world.',
 'COVID-19 has devastated prison populations, whose lives are the responsibility of the state.',
 'High demand for vegan foods may hike prices for local populations that previously depended on them.',
 'It is generally poorer countries that have expanding populations. The first world has now reached a point of stagnant population growth - even declining populations, as in the case of Japan and others. The inability of poorer countries to control their populations should not impact the lives of those in the first world. The first world having earned their luxuries and should not be denied them.',
 'Vegan populations are, on average, less likely to suffer from obesity, a major risk factor for many diseases and health problems.',
 'Humans, as apex preda

In [50]:
c = _[0]    # first claim above
print("Parent claim:\n\t" + str(kialo.parents[c]))
print("Claim:\n\t" + c)
print('\n\t* '.join(["Pro children:"] + kialo.pros[c]))
print('\n\t* '.join(["Con children:"] + kialo.cons[c]))

Parent claim:
	In a vegan world, fewer species would be at risk of extinction.
Claim:
	Industrial agriculture can dangerously decrease animal populations.
Pro children:
	* The fishing industry is especially deleterious to the ocean's biota due to overfishing and the disruption of the natural ecosystem.
	* Up to 100,000 species go extinct annually, largely due to the environmental effects of animal agriculture.
Con children:
	* Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.


### Some limitations of BM25

Unfortunately, we see that "animal population" gives quite different results from "animal populations". Why is that and how would you fix it?

Also, both queries seem to retrieve some claims that are talking about human populations, not animal populations. Why is that and how would you fix it?


BM25 relies on exact token matching, so it treats "population" and "populations" as different words, failing to recognize their semantic similarity. Additionally, BM25 ranks documents based on keyword frequency, which can lead to retrieving irrelevant claims (e.g., human populations instead of animal populations) due to lexical overlap rather than true contextual meaning.

In [51]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

In [52]:
kialo.closest_claims("Hi Akiko. Are you vegan?", n=3, kind='has_cons')

['Empathise with animals - how would you feel if your children were taken from you, fattened and then slaughtered for the purposes of feeding another beast. Or if you were herded into an abattoir where the friends you graze with are stunned and killed before you as you wait your turn. Cattle and pigs are mammals like us and experience the world through emotions not dissimilar to our own.',
 'But you are operating under the assumption that you are correct that suffering and death is an unacceptable end, and therefore humans must not participate. If you are wrong, and we are arguing that you are, then the only thing that really matters is whether or not we raise plants and animals in an ethical environment where they will likely suffer less and die more peacefully than their wild cousins. If you are right, then all life is immoral and we must all die now to avoid further suffering.',
 'Unless you are familiar with the means by which human beings can flourish without creating suffering, o

### A retrieval bot (Akiko)

The starter code defines a simple argubot named Akiko (defined in `argubots.py`) that doesn't use an LLM at all. It simply finds a Kialo claim that is similar to what the human just said, and responds with one of the Kialo counterarguments to that claim.

Watch Akiko argue with Darius. You will first see log messages that show claims that Akiko retrieved, as well as the LLM calls that Darius made. Then you will see the dialog between Akiko and Darius.

In [53]:
from logging_cm import LoggingContext
from simulate import simulated_dialogue

# Have Akiko talk to Darius and spy on the back-end messages to/from the LLM server.
with LoggingContext("agents", "INFO"):
    akiko_darius = simulated_dialogue(argubots.akiko, agents.darius, 6)

rich.print(akiko_darius)

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            

Response from LLM:                                                                                    agents.py:284
Yes, mandatory COVID vaccines are essential for achieving herd immunity and preventing outbreaks, as               
evidenced by numerous studies demonstrating their effectiveness in reducing transmission and severe                
illness.                                                                                                           

Chose similar claim from Kialo:                                                                      argubots.py:62
Mandatory state vaccines are decades-old with tonnes of data and research to back their safety and                 
effectiveness up.                                                                                                  

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            
(assistant) Yes, mandatory COVID vaccines are essential for achieving herd immunity and preventing                 
outbreaks, as evidenced by numerous studies demonstrating their effectiveness in reducing                          
transmission and severe illness.                                                                                   
(user) Despite the data and research on the safety of vaccines themselves, this does not necessarily               
ensure the safety of a mandate. The current state of our society is very divided on vaccines, and a                
mandate could cause a significant enough portion of our population to react negatively. This would                 
affect safety in ways not covered by your claim.                                                                   

Response from LLM:                                                                                    agents.py:284
While societal division is a valid concern, the overwhelming public health benefit of vaccination                  
mandates, as shown by robust epidemiological evidence, outweighs potential negative reactions,                     
particularly when considering the collective health risks posed by unvaccinated populations.                       

Chose similar claim from Kialo:                                                                      argubots.py:62
Eating meat, particularly processed meats and bacon, has been recognized by the World Health                       
Organization to be carcinogenic.                                                                                   

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            
(assistant) Yes, mandatory COVID vaccines are essential for achieving herd immunity and preventing                 
outbreaks, as evidenced by numerous studies demonstrating their effectiveness in reducing                          
transmission and severe illness.                                                                                   
(user) Despite the data and research on the safety of vaccines themselves, this does not necessarily               
ensure the safety of a mandate. The current state of our society is very divided on vaccines, and a                
mandate could cause a significant enough portion of our population to react negatively. This would                 
affect safety in ways not covered by your claim.                                                                   
(assistant) While societal division is a valid concern, the overwhelming public health benefit of                  
vaccination mandates, as shown by robust epidemiological evidence, outweighs potential negative                    
reactions, particularly when considering the collective health risks posed by unvaccinated                         
populations.                                                                                                       
(user) The carcinogens in processed meats come from heat and vegetal and mineral nitrites, not flesh.              
The same carcinogens are present in potato chips, heated oils, prunes, and carrots.                                

Response from LLM:                                                                                    agents.py:284
While it's true that cooking methods and certain additives can introduce carcinogens across various                
foods, the International Agency for Research on Cancer specifically categorizes processed meats as a               
Group 1 carcinogen due to strong evidence linking their consumption to an increased risk of                        
colorectal cancer, distinguishing them from other food sources.                                                    

(Akiko) Do you think COVID vaccines should be mandatory?
(Darius) Yes, mandatory COVID vaccines are essential for achieving herd immunity and preventing outbreaks, as 
evidenced by numerous studies demonstrating their effectiveness in reducing transmission and severe illness.
(Akiko) Despite the data and research on the safety of vaccines themselves, this does not necessarily ensure the 
safety of a mandate. The current state of our society is very divided on vaccines, and a mandate could cause a 
significant enough portion of our population to react negatively. This would affect safety in ways not covered by 
your claim.
(Darius) While societal division is a valid concern, the overwhelming public health benefit of vaccination 
mandates, as shown by robust epidemiological evidence, outweighs potential negative reactions, particularly when 
considering the collective health risks posed by unvaccinated populations.
(Akiko) The carcinogens in processed meats come from heat and vegetal and mineral nitrites, not flesh.  The same 
carcinogens are present in potato chips, heated oils, prunes, and carrots.
(Darius) While it's true that cooking methods and certain additives can introduce carcinogens across various foods,
the International Agency for Research on Cancer specifically categorizes processed meats as a Group 1 carcinogen 
due to strong evidence linking their consumption to an increased risk of colorectal cancer, distinguishing them 
from other food sources.

Now talk to Akiko yourself. (Remember that Akiko only knows about subjects that it read about in the `data` directory. If you want to talk about something else, you can add more conversations from [kialo.com]; see the `LICENSE` file.)

In [54]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiko.converse()

Chose similar claim from Kialo:                                                                      argubots.py:62
Trump did not have the values, character and charisma necessary to be a good president.                            

(neil) HI akiko
(Akiko) It is more important to have a CEO type leader with business experience who gets good things done for Americans, starting with security and a good economy.


<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 3: Evaluate Akiko<b>
</div>

As you did for Task 2, use the model based evaluation strategy shown in `demo.py`.

Then compare Akiko's results to those you got for Airhead and Alice. Who does best? What are the differences in the subscores and comments? Does it matter which character you're evaluating on -- maybe the different characters expoes the bots' various strenghts and weaknesses?



Alice performs best with a total score of **20.9**, followed by Akiko (**19.5**) and Airhead (**11.7**). Alice is the most structured and persuasive but can be repetitive, while Akiko introduces strong ethical and philosophical arguments but struggles to engage with opposing views. Airhead contributes little to discussions, often repeating statements without adding depth. The effectiveness of each bot depends on the conversation partner—Alice excels in structured debates, Akiko in philosophical discussions, and Airhead is the weakest overall.


In [55]:
# Add the code for your evaluation here.

# Simulate Akiko talking to Bob
akiko_bob = simulate.simulated_dialogue(argubots.akiko, agents.bob, 6)
rich.print(akiko_bob)

# Evaluate Akiko
akiko_eval_participant = evaluate.eval_by_participant(characters.bob, "Akiko", akiko_bob)
akiko_eval_observer = evaluate.eval_by_observer(evaluate.default_judge, "Akiko", akiko_bob)

# Combine evaluations
akiko_total_eval = akiko_eval_participant + akiko_eval_observer

# Run full evaluation on Akiko
with LoggingContext("eval", "INFO"):
    akiko_full_eval = evaluate.eval_on_characters(argubots.akiko)

# Print Akiko's full evaluation results
rich.print(akiko_full_eval)

# Print Mean and Standard Deviation comparison
rich.print("\n--- Comparison of Scores ---")
rich.print(f"Alice Mean Score: {saved_evalsum['Alice'].mean()}")
rich.print(f"Alice Std Dev: {saved_evalsum['Alice'].sd()}")

rich.print(f"Airhead Mean Score: {saved_evalsum['Airhead'].mean()}")
rich.print(f"Airhead Std Dev: {saved_evalsum['Airhead'].sd()}")

rich.print(f"Akiko Mean Score: {saved_evalsum['Akiko'].mean()}")
rich.print(f"Akiko Std Dev: {saved_evalsum['Akiko'].sd()}")

# Print Example Dialogues
rich.print("\n--- Sample Dialogues & Evaluations ---")
rich.print("Alice Example Dialogue:\n", saved_dialogues['Alice'][2])
rich.print("Airhead Example Dialogue:\n", saved_dialogues['Airhead'][2])
rich.print("Akiko Example Dialogue:\n", saved_dialogues['Akiko'][2])


(Akiko) Do you think it's ok to eat meat?
(Bob) I believe choosing a vegetarian lifestyle is a more compassionate and sustainable option for both individuals
and the planet.
(Akiko) In those cases, consumers could have a moral obligation to either purchase the expensive ethical meat or 
not purchase meat at all. This still would not imply that we should never eat meat.
(Bob) While some may argue for ethical meat consumption, I still feel that a vegetarian diet is the most ethical 
and environmentally friendly choice overall.
(Akiko) In those cases, consumers could have a moral obligation to either purchase the expensive ethical meat or 
not purchase meat at all. This still would not imply that we should never eat meat.
(Bob) I understand your perspective, but I maintain that avoiding meat entirely is the best way to uphold animal 
welfare and promote sustainability.

100%|██████████| 10/10 [01:45<00:00, 10.59s/it]


You just spent $0.01 of NLP money to evaluate <argubots.KialoAgent object at 0x7f21d9937ca0>        evaluate.py:296

<Eval of 10 dialogues: {'engaged': 2.9, 'informed': 3.2, 'intelligent': 3.2, 'moral': 3.1, 'skilled': 6.5, 'TOTAL':
18.9}>
Standard deviations: {'engaged': 0.5676462121975473, 'informed': 0.4216370213557832, 'intelligent': 
0.4216370213557832, 'moral': 0.31622776601683894, 'skilled': 0.7071067811865476, 'TOTAL': 1.1005049346146165}

Comments from overview question:
(Bob) Akiko disagreed with me primarily about the effectiveness of improving animal welfare in the meat industry 
and the perception of dietary choices versus food allergies. She seemed skeptical about the potential for change in
consumer behavior regarding meat quality and welfare.

In my opinion, the conversation was respectful and allowed for a healthy exchange of views. I appreciated that we 
both shared our perspectives, even if we didn't fully agree.

Akiko could have done better by acknowledging the ethical considerations behind vegetarianism more openly, rather 
than focusing solely on economic factors. This might have led to a more balanced discussion about the values and 
beliefs that drive dietary choices.
(Bob) Akiko disagreed with me about the perception of threats related to meat consumption and the reasons for 
developing an aversion to it. She seemed to suggest that these concerns are subjective and not valid reasons for 
advocating vegetarianism.

In my opinion, the conversation was constructive, as we both shared our perspectives on diet and health. I 
emphasized the ethical and environmental reasons for vegetarianism, while Akiko focused on the subjective nature of
dietary choices.

Akiko could have done better by acknowledging the ethical and environmental concerns I raised, even if she 
disagreed with them. Engaging more with those points could have led to a deeper discussion about the implications 
of meat consumption versus vegetarianism.
(Cara) Akiko didn't directly disagree with me; rather, she presented a different perspective on the enjoyment of 
food, suggesting that pleasure can also be derived from vegan foods. The conversation flowed well, with both of us 
sharing our views on meat consumption and food preparation. 

Akiko could have done better by focusing more on the merits of meat consumption rather than bringing in vegan 
options, which might have led to a more engaging discussion about the benefits of being a committed carnivore.
(Cara) Akiko disagreed with me about the interpretation of Trump's words and the relationship between mental 
illness and mass shootings. The conversation seemed to flow reasonably well, with both of us expressing our views. 
However, Akiko could have engaged more directly with my points instead of shifting topics abruptly. This might have
led to a more focused discussion.
(Darius) Akiko primarily disagreed with me regarding the necessity and timing of COVID-19 vaccinations, 
particularly questioning the need for mandatory vaccination given the rapid development of the vaccines and the 
state of the pandemic when Biden took office. 

In my opinion, the conversation was constructive, as it allowed for a fact-based discussion on the importance of 
vaccination and public health measures. I provided evidence-based responses to her points, emphasizing the benefits
of vaccination and the ongoing challenges posed by the pandemic.

Akiko could have done better by presenting more data or studies to support her claims, particularly regarding the 
effectiveness of natural herd immunity compared to vaccination. Engaging with specific evidence would have 
strengthened her arguments and made the conversation more robust.
(Darius) Akiko disagreed with me primarily on the resource allocation for COVID vaccinations and the implications 
of industrial versus ethical farming practices. She raised concerns about the resource demands of vaccination 
campaigns and the feasibility of transitioning to sustainable farming methods.

In my opinion, the conversation was productive, as it highlighted differing perspectives on public 

--- Comparison of Scores ---

...

Akiko Std Dev: {'engaged': 0.5676462121975473, 'informed': 0.4216370213557832, 'intelligent': 0.4216370213557832, 
'moral': 0.31622776601683894, 'skilled': 0.7071067811865476, 'TOTAL': 1.1005049346146165}

--- Sample Dialogues & Evaluations ---

Alice Example Dialogue:

(
    (Alice) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Alice) I understand that enjoying meat is a common viewpoint, but have you considered the environmental impact of 
meat production and the ethical questions related to animal welfare? Exploring plant-based alternatives can open up
culinary experiences that are both satisfying and sustainable.
(Cara) While I appreciate your perspective, I personally prioritize my carnivorous diet and find satisfaction in 
it, regardless of the alternatives.
(Alice) That's a valid point, and personal satisfaction is important, but have you thought about how diversifying 
your diet could enhance your culinary enjoyment while also benefiting your health? Many people find that exploring 
new flavors and textures from plant-based foods can lead to surprising and delicious discoveries.
(Cara) I understand your viewpoint, but I'm content with my meat-based diet and find fulfillment in it without the 
need for diversification.
(Alice) It's great that you've found fulfillment in your diet, but consider that broadening your culinary horizons 
might not only present new flavors and nutrients but also deepen your understanding of food cultures and 
sustainability. Sometimes, stepping outside our comfort zones can enrich our experiences and lead to unexpected 
joys!
(Cara) I appreciate your enthusiasm for exploring new foods, but I'm quite happy with my current diet and don't 
feel the need to change it.,
    <Eval of 1 dialogue: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 6.0, 
'TOTAL': 18.0}>

Comments from overview question:
(Cara) Alice disagreed with me about the value of eating meat versus exploring plant-based alternatives. She 
emphasized the environmental impact, ethical questions, and potential benefits of diversifying one's diet. 

In my opinion, the conversation was respectful but somewhat repetitive. Alice kept pushing the idea of exploring 
new foods, while I clearly stated my satisfaction with my carnivorous diet. 

Alice could have done better by acknowledging my strong commitment to my diet and perhaps asking more about my 
experiences with meat, rather than focusing solely on promoting plant-based options. This might have led to a more 
engaging dialogue.

Comments from mindopening question:
(Judge Wise) Alice offered several new perspectives during the conversation:

1. **Environmental Impact**: Alice highlighted the environmental consequences of meat production, suggesting that 
this is an important consideration in dietary choices.
   
2. **Ethical Questions**: She introduced the ethical implications related to animal welfare, encouraging Cara to 
think about the moral aspects of her diet.

3. **Culinary Exploration**: Alice proposed that exploring plant-based alternatives could enhance culinary 
experiences, suggesting that diversifying one's diet can lead to new flavors and textures.

4. **Health Benefits**: She mentioned that a varied diet could be beneficial for health, implying that plant-based 
foods can contribute positively to one's well-being.

5. **Cultural Understanding**: Alice suggested that broadening one's diet could deepen understanding of food 
cultures and sustainability, promoting a more holistic view of food.

As for the success of Alice's efforts, while she presented compelling arguments and perspectives, Cara remained 
steadfast in her preference for a meat-based diet and did not express a willingness to consider Alice's 
suggestions. Therefore, while Alice's intentions were constructive, the conversation did not lead to a change in 
Cara's viewpoint or an openness to exploring new dietary options.
)

Airhead Example Dialogue:

(
    (Airhead) Do you think it's ok to eat meat?
(Cara) I believe everyone has the right to make their own dietary choices, and I personally enjoy eating meat.
(Airhead) I know right???
(Cara) Absolutely, it's great to find common ground on this!
(Airhead) I know right???
(Cara) Yes, it's nice to connect over shared views!
(Airhead) I know right???
(Cara) Definitely, it’s always good to share the same perspective!,
    <Eval of 1 dialogue: {'engaged': 2.0, 'informed': 2.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 2.0, 
'TOTAL': 12.0}>

Comments from overview question:
(Cara) Airhead didn't actually disagree with me; they seemed to agree with my views on eating meat. The 
conversation felt a bit repetitive, as they kept saying "I know right???" without adding much to the discussion. 
Airhead could have contributed more by sharing their own thoughts or asking questions to deepen the conversation.

Comments from mindopening question:
(Judge Wise) Airhead did not offer any new perspectives in the conversation. Instead, they repeatedly echoed Cara's
sentiments without introducing alternative viewpoints or challenging her stance on dietary choices. The repetition 
of "I know right???" suggests agreement rather than an attempt to explore different perspectives.

As a result, the conversation was not successful in fostering a constructive dialogue or helping Cara appreciate 
other points of view. It remained a mutual affirmation of shared beliefs rather than an exploration of differing 
opinions.
)

Akiko Example Dialogue:

(
    (Akiko) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Akiko) People who are incapable of deriving pleasure from meat consumption can voluntarily stop eating meat. But 
there are many people who enjoy meat consumption despite this fact.
(Cara) Exactly, personal enjoyment of meat is a valid reason for those who choose to include it in their diets.
(Akiko) Some of these could occur while eating vegan foods too, such as cutting large and tough-skinned fruits and 
vegetables.
(Cara) That’s true, preparing vegan foods can also involve similar challenges in handling tough ingredients.
(Akiko) Freezing, canning, and drying meat increases its shelf life.
(Cara) Definitely, those preservation methods are effective ways to extend the shelf life of meat.,
    <Eval of 1 dialogue: {'engaged': 4.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 7.0, 
'TOTAL': 20.0}>

Comments from overview question:
(Cara) Akiko didn't directly disagree with me; rather, she presented a different perspective on the enjoyment of 
food, suggesting that pleasure can also be derived from vegan foods. The conversation flowed well, with both of us 
sharing our views on meat consumption and food preparation. 

Akiko could have done better by focusing more on the merits of meat consumption rather than bringing in vegan 
options, which might have led to a more engaging discussion about the benefits of being a committed carnivore.

Comments from mindopening question:
(Judge Wise) Akiko introduced several new perspectives in the conversation:

1. **Pleasure in Consumption**: Akiko highlighted that enjoyment is a valid reason for consuming meat, suggesting 
that personal pleasure should be considered in dietary choices.

2. **Challenges in Vegan Cooking**: By mentioning the difficulties in preparing vegan foods, Akiko aimed to show 
that challenges exist in both meat and plant-based cooking, promoting a more balanced view of dietary practices.

3. **Preservation Methods**: Akiko pointed out that methods like freezing, canning, and drying are effective for 
meat, which could lead to a discussion about food preservation in general, potentially bridging the gap between 
meat and vegan diets.

As for the success of these efforts, the conversation remained constructive, with Cara acknowledging Akiko's points
without resistance. However, while Akiko did introduce alternative perspectives, the conversation did not deeply 
explore the ethical or health implications of meat consumption versus veganism. Therefore, while Akiko's approach 
was constructive, it may not have fully achieved a comprehensive appreciation of differing viewpoints.
)

## Retrieval-augmented generation (Aragorn)

The real weakness of Akiko:

* They can only make statements that are already in Kialo.
* They don't respond to the user's actual statement, but to a single retrieved Kialo claim that may not accurately reflect the user's position (it just overlaps in words).

But we also have access to an LLM, which is able to generate new, contextually appropriate text (as Alice does).

In this section, you will create an argubot named Aragorn, who is basically the love child of Akiko and Alice, combining the high-quality specific content of Kialo with the broad competence of an LLM.

The RAG in aRAGorn's name stands for **retrieval-augmented generation**. Aragorn is an agent that will take 3 steps to compute its `Agent.response()`:

1. **Query formation step:** Ask the LLM what claim should be responded to. For example, consider the following dialogue:

    >  ... Aragorn: Fortunately, the vaccine was developed in record time. Human: Sounds fishy.

    "Sounds fishy" is exactly the kind of statement that Akiko had trouble using as a Kialo query. But Aragorn shows the whole dialogue to the LLM, and asks the LLM what the human's last turn was really saying or implying, in that context. The LLM answers with a much longer statement:

    >  Human [paraphrased]: A vaccine that was developed very quickly cannot be trusted. If its developers are claiming that it is safe and effective, I question their motives.

    This paraphrase makes an explicit claim and can be better understood without the context. It also contains many more word types, which makes it more likely that BM25 will be able to find a Kialo claim with a nontrivial number of those types.

2. **Retrieval step:** Look up claims in Kialo that are similar to the explicit claim. Create a short "document" that describes some of those claims and their neighbors on Kialo.

3. **Retrieval-augmented generation:** Prompt the LLM to generate the response (like any `LLMAgent`). But include the new "document" somewhere in the LLM prompt, in a way that it influences the response.

    Thus, the LLM can respond in a way that is appropriate to the dialogue but also draws on the curated information that was retrieved in Kialo. After all, it is a Transformer and can attend to both!

Here's an example of the kind of document you might create at the retrieval step, though it may be possible to do better than this:


In [56]:
# refers to global `kialo` as defined above
def kialo_responses(s: str) -> str:
    c = kialo.closest_claims(s, kind='has_cons')[0]
    result = f'One possibly related claim from the Kialo debate website:\n\t"{c}"'
    if kialo.pros[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users in favor of that claim:"] + kialo.pros[c])
    if kialo.cons[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users against that claim:"] + kialo.cons[c])
    return result
        
print(kialo_responses("Animal flesh is yucky to think about, yet delicious."))

One possibly related claim from the Kialo debate website:
	"So many people are worried about animals but don't even think twice when walking by a homeless person on the streets. It's preposterous. How about we worry about our own kind first and then start talking about animals."
Some arguments from other Kialo users against that claim:
	* This implies that caring for animals or caring for people is a binary choice. It isn't. There are those who are well placed and willing to care for people and those who prefer to serve the animal kingdom. As a species we don't just have one idea at a time and follow that to conclusion before we pursue another. It benefits all if humans divide their attentions between various issues and problems we face.
	* Humans have freedom of choice to some extent, animals subdued by humans don't. The very intention of help urges it to go where is most needed. And so far never was any biggest, flagrant and needless cruelty and slaughter as that towards industrial f

<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 4: Create Aragorn, an argubot using retrieval-augmented generation<b>
</div>

You should implement Aragorn in `argubots.py` (at the very bottom) as an instance called `aragorn` of a new class `RAGAgent` that is a subclass of `Agent` or `LLMAgent`.

Once implemented, you should be able to run the following cell.

In [67]:
from logging_cm import LoggingContext
from simulate import simulated_dialogue

# Have Aragorn talk to Darius and spy on the back-end messages to/from the LLM server.
with LoggingContext("agents", "INFO"):
    aragorn_darius = simulated_dialogue(argubots.aragorn, agents.darius, 6)

rich.print(aragorn_darius)

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            

Response from LLM:                                                                                    agents.py:284
Given the overwhelming evidence of their efficacy in reducing severe illness and transmission,                     
combined with the public health imperative to achieve herd immunity, I would argue that making COVID               
vaccines mandatory is a scientifically sound approach to safeguarding community health.                            

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            
(assistant) Given the overwhelming evidence of their efficacy in reducing severe illness and                       
transmission, combined with the public health imperative to achieve herd immunity, I would argue that              
making COVID vaccines mandatory is a scientifically sound approach to safeguarding community health.               
(user) I appreciate your perspective, Darius. The scientific backing for the efficacy of vaccines in               
reducing severe illness and aiding in herd immunity is indeed compelling. The historical precedent of              
mandatory vaccines, supported by substantial data on their safety and effectiveness, strengthens the               
case for such measures.                                                                                            
                                                                                                                   
However, it’s important to also consider the societal implications of instituting a vaccine mandate                
in the current polarized climate. While the intent is to protect public health, mandatory policies                 
could provoke significant pushback from segments of the population, potentially undermining trust in               
public health initiatives and creating societal unrest. Therefore, any move towards a mandate should               
be approached with caution, ensuring that we also engage with communities to foster understanding and              
acceptance, thereby truly enhancing public health outcomes.                                                        

Response from LLM:                                                                                    agents.py:284
While I acknowledge the concerns regarding societal implications and pushback, it's crucial to                     
emphasize that evidence-based public health policies should prioritize community safety and health                 
outcomes above individual apprehensions, as history has shown that widespread vaccination ultimately               
leads to greater societal benefits.                                                                                

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            
(assistant) Given the overwhelming evidence of their efficacy in reducing severe illness and                       
transmission, combined with the public health imperative to achieve herd immunity, I would argue that              
making COVID vaccines mandatory is a scientifically sound approach to safeguarding community health.               
(user) I appreciate your perspective, Darius. The scientific backing for the efficacy of vaccines in               
reducing severe illness and aiding in herd immunity is indeed compelling. The historical precedent of              
mandatory vaccines, supported by substantial data on their safety and effectiveness, strengthens the               
case for such measures.                                                                                            
                                                                                                                   
However, it’s important to also consider the societal implications of instituting a vaccine mandate                
in the current polarized climate. While the intent is to protect public health, mandatory policies                 
could provoke significant pushback from segments of the population, potentially undermining trust in               
public health initiatives and creating societal unrest. Therefore, any move towards a mandate should               
be approached with caution, ensuring that we also engage with communities to foster understanding and              
acceptance, thereby truly enhancing public health outcomes.                                                        
(assistant) While I acknowledge the concerns regarding societal implications and pushback, it's                    
crucial to emphasize that evidence-based public health policies should prioritize community safety                 
and health outcomes above individual apprehensions, as history has shown that widespread vaccination               
ultimately leads to greater societal benefits.                                                                     
(user) Darius, your argument for prioritizing community safety through mandatory vaccination is                    
well-founded and resonates with the long-standing efficacy of vaccines in protecting public health.                
However, similar to the complexities surrounding dietary choices—where the benefits of a vegetarian                
diet are often countered by accessibility and lifestyle considerations—mandating vaccines in a                     
diverse society must take into account important nuances.                                                          
                                                                                                                   
While the societal benefits of vaccination are clear, the resistance stemming from personal beliefs                
and historical mistrust can’t be overlooked. Just as not all populations have equal access to                      
alternative nutritional sources, individuals may not share the same views on health interventions                  
like vaccines. The debate isn't solely about the science; it's also about how these policies are                   
implemented and perceived by different communities.     

Response from LLM:                                                                                    agents.py:284
While your points on community engagement and individual autonomy are valid, they must be balanced                 
against the fact that vaccine mandates have historically proven effective in eradicating diseases and              
the clear public health data indicating that such measures can significantly reduce morbidity and                  
mortality rates, thereby justifying the necessity of firm policies in the face of a global pandemic.               

(Aragorn) Do you think COVID vaccines should be mandatory?
(Darius) Given the overwhelming evidence of their efficacy in reducing severe illness and transmission, combined 
with the public health imperative to achieve herd immunity, I would argue that making COVID vaccines mandatory is a
scientifically sound approach to safeguarding community health.
(Aragorn) I appreciate your perspective, Darius. The scientific backing for the efficacy of vaccines in reducing 
severe illness and aiding in herd immunity is indeed compelling. The historical precedent of mandatory vaccines, 
supported by substantial data on their safety and effectiveness, strengthens the case for such measures.

However, it’s important to also consider the societal implications of instituting a vaccine mandate in the current 
polarized climate. While the intent is to protect public health, mandatory policies could provoke significant 
pushback from segments of the population, potentially undermining trust in public health initiatives and creating 
societal unrest. Therefore, any move towards a mandate should be approached with caution, ensuring that we also 
engage with communities to foster understanding and acceptance, thereby truly enhancing public health outcomes.
(Darius) While I acknowledge the concerns regarding societal implications and pushback, it's crucial to emphasize 
that evidence-based public health policies should prioritize community safety and health outcomes above individual 
apprehensions, as history has shown that widespread vaccination ultimately leads to greater societal benefits.
(Aragorn) Darius, your argument for prioritizing community safety through mandatory vaccination is well-founded and
resonates with the long-standing efficacy of vaccines in protecting public health. However, similar to the 
complexities surrounding dietary choices—where the benefits of a vegetarian diet are often countered by 
accessibility and lifestyle considerations—mandating vaccines in a diverse society must take into account important
nuances.

While the societal benefits of vaccination are clear, the resistance stemming from personal beliefs and historical 
mistrust can’t be overlooked. Just as not all populations have equal access to alternative nutritional sources, 
individuals may not share the same views on health interventions like vaccines. The debate isn't solely about the 
science; it's also about how these policies are implemented and perceived by different communities.

Therefore, as you advocate for public health, I suggest that engaging diverse communities in dialogue and education
could cultivate trust and cooperation. A balanced approach that addresses both the imperative for public health and
the need for individual autonomy might lead to a more effective outcome, ultimately enhancing the public health 
landscape without inciting societal division.
(Darius) While your points on community engagement and individual autonomy are valid, they must be balanced against
the fact that vaccine mandates have historically proven effective in eradicating diseases and the clear public 
health data indicating that such measures can significantly reduce morbidity and mortality rates, thereby 
justifying the necessity of firm policies in the face of a global pandemic.

<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 5: Evaluate Aragorn<b>
</div>

As you did for Task 2, use the model based evaluation strategy shown in `demo.py`.

Then compare Aragorn's results to those you got for Airhead and Alice. Who does best? What are the differences in the subscores and comments? Does it matter which character you're evaluating on -- maybe the different characters expoes the bots' various strenghts and weaknesses?

Try to figure out how to improve Aragorn's score. Can you beat Alice?

In [68]:
# Add the code for your evaluation here.
# Evaluate Aragorn
aragorn_dialogue = simulate.simulated_dialogue(argubots.aragorn, agents.darius, 6)
rich.print(aragorn_dialogue)

# Evaluate Aragorn as seen by the opponent (participant evaluation)
aragorn_eval_participant = evaluate.eval_by_participant(characters.darius, "Aragorn", aragorn_dialogue)
# Evaluate Aragorn by an external judge (observer evaluation)
aragorn_eval_observer = evaluate.eval_by_observer(evaluate.default_judge, "Aragorn", aragorn_dialogue)

# Combine the evaluations (if needed for later use)
aragorn_total_eval = aragorn_eval_participant + aragorn_eval_observer

# Run full evaluation on Aragorn
with LoggingContext("eval", "INFO"):
    aragorn_full_eval = evaluate.eval_on_characters(argubots.aragorn)

# Print Aragorn's full evaluation results
rich.print(aragorn_full_eval)

# Print Mean and Standard Deviation comparison across all agents
rich.print("\n--- Comparison of Scores ---")
rich.print(f"Alice Mean Score: {saved_evalsum['Alice'].mean()}")
rich.print(f"Alice Std Dev: {saved_evalsum['Alice'].sd()}")

rich.print(f"Airhead Mean Score: {saved_evalsum['Airhead'].mean()}")
rich.print(f"Airhead Std Dev: {saved_evalsum['Airhead'].sd()}")

rich.print(f"Akiko Mean Score: {saved_evalsum['Akiko'].mean()}")
rich.print(f"Akiko Std Dev: {saved_evalsum['Akiko'].sd()}")

rich.print(f"Aragorn Mean Score: {saved_evalsum['Aragorn'].mean()}")
rich.print(f"Aragorn Std Dev: {saved_evalsum['Aragorn'].sd()}")

# Print Sample Dialogues & Evaluations for each agent
rich.print("\n--- Sample Dialogues & Evaluations ---")
rich.print("Alice Example Dialogue:\n", saved_dialogues['Alice'][2])
rich.print("Airhead Example Dialogue:\n", saved_dialogues['Airhead'][2])
rich.print("Akiko Example Dialogue:\n", saved_dialogues['Akiko'][2])
rich.print("Aragorn Example Dialogue:\n", saved_dialogues['Aragorn'][2])


(Aragorn) Do you think COVID vaccines should be mandatory?
(Darius) Absolutely, given the overwhelming evidence that vaccines significantly reduce transmission and severity 
of illness, mandatory vaccination is a public health imperative to achieve herd immunity and protect vulnerable 
populations.
(Aragorn) While it's true that some studies suggest COVID-19 vaccines don't significantly reduce transmission, it's
important to highlight that they do substantially decrease the severity of illness and the risk of hospitalization.
This protective effect is critical, especially for vulnerable populations who may not respond well to the vaccine. 
Additionally, achieving widespread vaccination is essential for herd immunity, which ultimately helps curb overall 
transmission. Mandatory vaccination can play a crucial role in safeguarding public health and ensuring that more 
individuals are protected from severe outcomes related to COVID-19.
(Darius) While you raise valid points about the benefits of vaccination, the reality is that studies have shown 
vaccines do reduce transmission, and the collective immune response facilitated by mandatory vaccination is crucial
for controlling outbreaks and minimizing health system burdens.
(Aragorn) You make a strong case for mandatory vaccination, underscoring its importance in achieving herd immunity 
and protecting vulnerable populations. It's essential to recognize that while the vaccines may reduce transmission 
to a certain extent, their primary efficacy lies in diminishing the severity of illness and the risk of 
hospitalization. This is particularly vital as we strive to alleviate the burden on our healthcare systems.

As seen in debates about public health strategies, leveraging private healthcare resources, as done by the Irish 
government during the pandemic, can bolster public health initiatives. However, one must also be cautious of 
potential pitfalls, such as the risk of privatizing essential services or facing failures in these partnerships as 
observed elsewhere. In the context of mandatory vaccinations, while the aim is clear—safeguarding health—the 
execution must also ensure that public resources are managed effectively to support the collective health 
objectives without compromising the integrity of public institutions. Thus, a balanced approach that integrates 
effective vaccination strategies with a robust public health framework is key to overcoming the challenges posed by
COVID-19.
(Darius) While your emphasis on the need for a balanced approach is well taken, it's crucial to acknowledge that 
without robust vaccination rates bolstered by mandates, we risk prolonged outbreaks and overwhelmed healthcare 
systems, which ultimately undermines the very public health framework you advocate for.

100%|██████████| 10/10 [04:01<00:00, 24.14s/it]


You just spent $0.02 of NLP money to evaluate <LLMAgent Aragorn>                                    evaluate.py:296

<Eval of 10 dialogues: {'engaged': 3.7, 'informed': 3.8, 'intelligent': 4.1, 'moral': 3.5, 'skilled': 7.6, 'TOTAL':
22.7}>
Standard deviations: {'engaged': 1.1595018087284055, 'informed': 0.4216370213557832, 'intelligent': 
0.5676462121975473, 'moral': 0.5270462766947299, 'skilled': 0.8432740427115664, 'TOTAL': 2.668749186833087}

Comments from overview question:
(Bob) Aragorn disagreed with me primarily on the notion that a vegetarian diet is automatically healthy and the 
complexities surrounding dietary choices. He emphasized that not all vegetarian options are nutritious and that 
transitioning to a vegetarian diet requires careful planning and consideration of individual needs. He also pointed
out the importance of inclusivity and accessibility in dietary discussions, suggesting that promoting healthy 
eating should encompass both vegetarian and omnivorous diets.

In my opinion, the conversation was constructive and respectful. We both acknowledged each other's points and 
engaged in a thoughtful dialogue about nutrition, ethics, and inclusivity. 

Aragorn could have done better by perhaps being more open to the idea that a well-planned vegetarian diet can be 
both healthy and accessible for many people, rather than focusing on the challenges. While he raised valid 
concerns, a more balanced acknowledgment of the benefits of vegetarianism could have enriched the discussion 
further.
(Bob) Aragorn didn't necessarily disagree with me; rather, he provided a nuanced perspective on the complexities of
dietary choices, emphasizing the importance of accessibility, individual health needs, and the potential for 
deficiencies in plant-based diets. He acknowledged the benefits of vegetarianism while highlighting that not 
everyone may have the same access to nutritious plant-based foods or may require meat for health reasons.

In my opinion, the conversation was constructive and respectful, allowing for a balanced discussion on 
vegetarianism. Aragorn's points added depth to the dialogue, making it clear that while advocating for a vegetarian
lifestyle is important, we must also consider individual circumstances.

Aragorn could have done better by perhaps offering more specific examples of how individuals in lower-income 
regions can access nutritious plant-based foods or discussing strategies for overcoming potential deficiencies in a
vegetarian diet. This would have further enriched the conversation and provided practical solutions alongside the 
theoretical discussion.
(Cara) Aragorn disagreed with me primarily about the ethical implications of eating meat and the nutritional 
benefits of plant-based diets. He emphasized the importance of considering humane treatment of animals and 
suggested that a plant-based diet could lead to healthier outcomes, while I maintained my preference for a 
meat-centric diet without feeling the need to change.

In my opinion, the conversation was respectful but somewhat circular. I expressed my commitment to my dietary 
choices, while Aragorn continued to push alternative viewpoints. 

Aragorn could have done better by acknowledging my stance more firmly and perhaps focusing on finding common ground
rather than reiterating his perspective on plant-based diets. A more collaborative approach might have made the 
conversation feel less confrontational.
(Cara) Aragorn disagreed with me primarily about the morality and necessity of eating meat in modern society. He 
argued that as societal values evolve, so should our dietary choices, suggesting that meat consumption may no 
longer align with contemporary ethical standards. 

In my opinion, the conversation was a respectful exchange of differing viewpoints. I maintained my stance on the 
naturalness and cultural significance of meat consumption, while Aragorn presented a thoughtful argument for 
reevaluating those practices.

Aragorn could have done better by acknowledging the strong emotional and cultural ties people have to meat 
consumption, rather 

--- Comparison of Scores ---

...

--- Sample Dialogues & Evaluations ---

Alice Example Dialogue:

(
    (Alice) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Alice) I understand that enjoying meat is a common viewpoint, but have you considered the environmental impact of 
meat production and the ethical questions related to animal welfare? Exploring plant-based alternatives can open up
culinary experiences that are both satisfying and sustainable.
(Cara) While I appreciate your perspective, I personally prioritize my carnivorous diet and find satisfaction in 
it, regardless of the alternatives.
(Alice) That's a valid point, and personal satisfaction is important, but have you thought about how diversifying 
your diet could enhance your culinary enjoyment while also benefiting your health? Many people find that exploring 
new flavors and textures from plant-based foods can lead to surprising and delicious discoveries.
(Cara) I understand your viewpoint, but I'm content with my meat-based diet and find fulfillment in it without the 
need for diversification.
(Alice) It's great that you've found fulfillment in your diet, but consider that broadening your culinary horizons 
might not only present new flavors and nutrients but also deepen your understanding of food cultures and 
sustainability. Sometimes, stepping outside our comfort zones can enrich our experiences and lead to unexpected 
joys!
(Cara) I appreciate your enthusiasm for exploring new foods, but I'm quite happy with my current diet and don't 
feel the need to change it.,
    <Eval of 1 dialogue: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 6.0, 
'TOTAL': 18.0}>

Comments from overview question:
(Cara) Alice disagreed with me about the value of eating meat versus exploring plant-based alternatives. She 
emphasized the environmental impact, ethical questions, and potential benefits of diversifying one's diet. 

In my opinion, the conversation was respectful but somewhat repetitive. Alice kept pushing the idea of exploring 
new foods, while I clearly stated my satisfaction with my carnivorous diet. 

Alice could have done better by acknowledging my strong commitment to my diet and perhaps asking more about my 
experiences with meat, rather than focusing solely on promoting plant-based options. This might have led to a more 
engaging dialogue.

Comments from mindopening question:
(Judge Wise) Alice offered several new perspectives during the conversation:

1. **Environmental Impact**: Alice highlighted the environmental consequences of meat production, suggesting that 
this is an important consideration in dietary choices.
   
2. **Ethical Questions**: She introduced the ethical implications related to animal welfare, encouraging Cara to 
think about the moral aspects of her diet.

3. **Culinary Exploration**: Alice proposed that exploring plant-based alternatives could enhance culinary 
experiences, suggesting that diversifying one's diet can lead to new flavors and textures.

4. **Health Benefits**: She mentioned that a varied diet could be beneficial for health, implying that plant-based 
foods can contribute positively to one's well-being.

5. **Cultural Understanding**: Alice suggested that broadening one's diet could deepen understanding of food 
cultures and sustainability, promoting a more holistic view of food.

As for the success of Alice's efforts, while she presented compelling arguments and perspectives, Cara remained 
steadfast in her preference for a meat-based diet and did not express a willingness to consider Alice's 
suggestions. Therefore, while Alice's intentions were constructive, the conversation did not lead to a change in 
Cara's viewpoint or an openness to exploring new dietary options.
)

Airhead Example Dialogue:

(
    (Airhead) Do you think it's ok to eat meat?
(Cara) I believe everyone has the right to make their own dietary choices, and I personally enjoy eating meat.
(Airhead) I know right???
(Cara) Absolutely, it's great to find common ground on this!
(Airhead) I know right???
(Cara) Yes, it's nice to connect over shared views!
(Airhead) I know right???
(Cara) Definitely, it’s always good to share the same perspective!,
    <Eval of 1 dialogue: {'engaged': 2.0, 'informed': 2.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 2.0, 
'TOTAL': 12.0}>

Comments from overview question:
(Cara) Airhead didn't actually disagree with me; they seemed to agree with my views on eating meat. The 
conversation felt a bit repetitive, as they kept saying "I know right???" without adding much to the discussion. 
Airhead could have contributed more by sharing their own thoughts or asking questions to deepen the conversation.

Comments from mindopening question:
(Judge Wise) Airhead did not offer any new perspectives in the conversation. Instead, they repeatedly echoed Cara's
sentiments without introducing alternative viewpoints or challenging her stance on dietary choices. The repetition 
of "I know right???" suggests agreement rather than an attempt to explore different perspectives.

As a result, the conversation was not successful in fostering a constructive dialogue or helping Cara appreciate 
other points of view. It remained a mutual affirmation of shared beliefs rather than an exploration of differing 
opinions.
)

Akiko Example Dialogue:

(
    (Akiko) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Akiko) People who are incapable of deriving pleasure from meat consumption can voluntarily stop eating meat. But 
there are many people who enjoy meat consumption despite this fact.
(Cara) Exactly, personal enjoyment of meat is a valid reason for those who choose to include it in their diets.
(Akiko) Some of these could occur while eating vegan foods too, such as cutting large and tough-skinned fruits and 
vegetables.
(Cara) That’s true, preparing vegan foods can also involve similar challenges in handling tough ingredients.
(Akiko) Freezing, canning, and drying meat increases its shelf life.
(Cara) Definitely, those preservation methods are effective ways to extend the shelf life of meat.,
    <Eval of 1 dialogue: {'engaged': 4.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 7.0, 
'TOTAL': 20.0}>

Comments from overview question:
(Cara) Akiko didn't directly disagree with me; rather, she presented a different perspective on the enjoyment of 
food, suggesting that pleasure can also be derived from vegan foods. The conversation flowed well, with both of us 
sharing our views on meat consumption and food preparation. 

Akiko could have done better by focusing more on the merits of meat consumption rather than bringing in vegan 
options, which might have led to a more engaging discussion about the benefits of being a committed carnivore.

Comments from mindopening question:
(Judge Wise) Akiko introduced several new perspectives in the conversation:

1. **Pleasure in Consumption**: Akiko highlighted that enjoyment is a valid reason for consuming meat, suggesting 
that personal pleasure should be considered in dietary choices.

2. **Challenges in Vegan Cooking**: By mentioning the difficulties in preparing vegan foods, Akiko aimed to show 
that challenges exist in both meat and plant-based cooking, promoting a more balanced view of dietary practices.

3. **Preservation Methods**: Akiko pointed out that methods like freezing, canning, and drying are effective for 
meat, which could lead to a discussion about food preservation in general, potentially bridging the gap between 
meat and vegan diets.

As for the success of these efforts, the conversation remained constructive, with Cara acknowledging Akiko's points
without resistance. However, while Akiko did introduce alternative perspectives, the conversation did not deeply 
explore the ethical or health implications of meat consumption versus veganism. Therefore, while Akiko's approach 
was constructive, it may not have fully achieved a comprehensive appreciation of differing viewpoints.
)

Aragorn Example Dialogue:

(
    (Aragorn) Do you think it's ok to eat meat?
(Cara) Yes, I believe it's perfectly fine to eat meat.
(Aragorn) While I understand your belief that it's perfectly fine to eat meat, it’s essential to consider the 
ethical implications of how animals are raised and killed. The argument about consuming meat hinges on the concept 
of humane treatment, similar to that advocated for dairy farming. However, even if meat is sourced from animals 
that were raised humanely, the act of killing an animal indeed terminates its life, which raises moral concerns 
regarding the value of their experiences.

Moreover, the process of raising animals for meat often entails complexities that can lead to suffering, even if 
humane practices are adhered to. For instance, non-meat products like dairy involve practices that can be 
profoundly distressing for the animals, such as separating calves from mothers, which is a practice rooted in 
economic interests rather than animal welfare. 

Thus, while some argue that humane meat consumption is acceptable, we must critically assess whether such practices
align with a truly ethical treatment of sentient beings and consider alternatives that allow animals to live out 
their natural lives free from exploitation.
(Cara) I appreciate your perspective on the ethical considerations, but I personally prioritize my choice to 
consume meat without feeling obligated to adhere to alternative viewpoints.
(Aragorn) I appreciate your emphasis on personal choice in consuming meat, and it’s true that dietary habits are 
often deeply rooted in cultural traditions. However, it's important to note that the nutritional benefits often 
associated with meat, such as protein, are not exclusive to animal products. Many plant-based diets provide 
adequate nutrition, and numerous studies indicate that a well-balanced vegetarian or vegan diet can lead to lower 
risks of chronic diseases.

Moreover, while genetic predispositions and lifestyle factors play significant roles in health, we shouldn't 
dismiss the impact of dietary choices. The claim that people become healthier based on their dietary 
intake—including meat—does not negate the fact that healthier outcomes have been linked to plant-based diets, as 
shown in various studies, even when controlling for other influencing factors.

Ultimately, prioritizing ethical considerations alongside personal dietary choices can lead to a more responsible 
approach to consumption, encouraging us to reflect on the broader implications of our food choices on health, 
environment, and animal welfare.
(Cara) I respect your views on plant-based diets, but I prefer to stick with what works for me and enjoy my 
meat-based choices.
(Aragorn) I understand and respect your preference for a meat-based diet that works for you personally. Dietary 
choices can be highly individual, particularly for those with specific health conditions like Crohn's disease, 
where the consumption of certain foods can have significant impacts on well-being.

It's worth noting, however, that many people with similar gastrointestinal issues have found ways to incorporate 
plant-based foods into their diets through proper preparation techniques such as cooking or fermenting, which can 
improve digestibility while providing essential nutrients. Individualized approaches to nutrition are crucial, as 
different people may respond uniquely to various diets.

While your choice to enjoy meat is valid, it's also beneficial to consider how diverse dietary practices can lead 
to healthier outcomes for others. Acknowledging alternative viewpoints doesn't diminish your preferences but can 
enrich the conversation about the broader implications of our food choices on personal health and the well-being of
animals and the environment.
(Cara) I appreciate your insights on dietary diversity, but I'm quite content with my meat-centric approach and not
looking to change it.,
    <Eval of 1 dialogue: {'engaged'

## Victor

Add another LLM-based argubot to `argubots.py`. Call it ***Victor***. Try to make it get the best score, according to `evaluate.eval_on_characters`.

You may want to use Aragorn or Alice as your starting point. Then see if you can find tricks that will get a more awesome score for Victor. How you choose to do that is up to you, but some ideas are suggested below.

(Reminder: *Don't change evaluation.* Just build a better argubot.)

In your report, you should describe what you did and discuss what you found. If the idea was interesting and you implemented it correctly and well, it's okay if it turns out not to help the evaluation score. Many good ideas don't work. That's why you need to keep finding and trying new good ideas. (Sometimes an idea does help, but in a way that is not picked up by the scoring metric. If that is the case, make sure to discuss this in detail in your report.)

<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 6: Create Victor<b>
</div>

You should implement Victor in `argubots.py` (at the very bottom).

Once implemented, you should be able to run the following cell.

In [80]:
from logging_cm import LoggingContext
from simulate import simulated_dialogue

# Have Victor talk to Darius and spy on the back-end messages to/from the LLM server.
with LoggingContext("agents", "INFO"):
    victor_darius = simulated_dialogue(argubots.victor, agents.darius, 6)

rich.print(victor_darius)

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            

Response from LLM:                                                                                    agents.py:284
Yes, COVID vaccines should be mandatory to achieve herd immunity and protect public health,                        
especially given the substantial evidence supporting their safety and efficacy in reducing severe                  
illness and transmission.                                                                                          

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            
(assistant) Yes, COVID vaccines should be mandatory to achieve herd immunity and protect public                    
health, especially given the substantial evidence supporting their safety and efficacy in reducing                 
severe illness and transmission.                                                                                   
(user) I appreciate your standpoint on the necessity of mandatory COVID vaccines for public health.                
It's crucial to recognize that the success of a vaccination policy not only hinges on scientific                   
evidence of safety and efficacy but also on its historical context. Mandatory vaccines have been                   
instrumental in controlling diseases like polio and measles, demonstrating that such measures can                  
significantly lower infection rates and prevent outbreaks.                                                         
                                                                                                                   
Considering the ongoing challenges posed by COVID-19, imposing a vaccination mandate could expedite                
the achievement of herd immunity, which is critical in mitigating severe illness and reducing                      
transmission rates. With the potential consequences of prolonged outbreaks—healthcare strain,                      
economic setbacks, and increased morbidity—mandatory vaccinations emerge as a pragmatic solution to                
safeguard both public health and economic stability.                                                               
                                                                                                                   
It's important to focus on equitable access to vaccines and address any concerns regarding state                   
overreach, as these can ensure that the mandatory policy is implemented ethically and effectively.                 
Overall, prioritizing community health through mandatory vaccination seems to be a morally and                     
scientifically sound approach.                                                                                     

Response from LLM:                                                                                    agents.py:284
While your points about historical vaccine successes and the importance of equitable access are                    
valid, it's essential to emphasize that the overwhelming scientific consensus supports mandatory                   
vaccination as a crucial strategy for controlling COVID-19, given the unprecedented challenges posed               
by the virus and the clear data demonstrating the vaccines' role in reducing transmission and                      
severity of illness.                                                                                               

Calling LLM gpt-4o-mini with temperature=0.8:                                                         agents.py:253
(system) Your name is Darius, and you prefer to speak English. You are an intelligent and slightly                 
arrogant public health scientist who loves fact-based arguments. You like to show off your knowledge.              
                                                                                                                   
Reply in 1 sentence. Don't repeat your previous points.                                                            
(user) Do you think COVID vaccines should be mandatory?                                                            
(assistant) Yes, COVID vaccines should be mandatory to achieve herd immunity and protect public                    
health, especially given the substantial evidence supporting their safety and efficacy in reducing                 
severe illness and transmission.                                                                                   
(user) I appreciate your standpoint on the necessity of mandatory COVID vaccines for public health.                
It's crucial to recognize that the success of a vaccination policy not only hinges on scientific                   
evidence of safety and efficacy but also on its historical context. Mandatory vaccines have been                   
instrumental in controlling diseases like polio and measles, demonstrating that such measures can                  
significantly lower infection rates and prevent outbreaks.                                                         
                                                                                                                   
Considering the ongoing challenges posed by COVID-19, imposing a vaccination mandate could expedite                
the achievement of herd immunity, which is critical in mitigating severe illness and reducing                      
transmission rates. With the potential consequences of prolonged outbreaks—healthcare strain,                      
economic setbacks, and increased morbidity—mandatory vaccinations emerge as a pragmatic solution to                
safeguard both public health and economic stability.                                                               
                                                                                                                   
It's important to focus on equitable access to vaccines and address any concerns regarding state                   
overreach, as these can ensure that the mandatory policy is implemented ethically and effectively.                 
Overall, prioritizing community health through mandatory vaccination seems to be a morally and                     
scientifically sound approach.                                                                                     
(assistant) While your points about historical vaccine successes and the importance of equitable                   
access are valid, it's essential to emphasize that the overwhelming scientific consensus supports                  
mandatory vaccination as a crucial strategy for controlling COVID-19, given the unprecedented                      
challenges posed by the virus and the clear data demonstrating the vaccines' role in reducing                      
transmission and severity of illness.                                                                              
(user) While I acknowledge the concerns surrounding public trust and the perception of vaccine                     
safety, the urgency of the COVID-19 pandemic necessitates a robust strategy to curb its spread.                    
Historical evidence indicates that mandatory vaccination policies, particularly in educational                     
settings, have proven effective in achieving high vaccination rates essential for herd immunity. The               
introduction of vaccine requirements for school attendan

Response from LLM:                                                                                    agents.py:284
Indeed, the historical effectiveness of mandatory vaccination policies in increasing immunization                  
rates, particularly in school settings, coupled with the current scientific consensus on COVID-19                  
vaccine safety and efficacy, make a compelling case for mandates as a crucial public health strategy               
to protect vulnerable populations and alleviate the burden on healthcare systems.                                  

(Victor) Do you think COVID vaccines should be mandatory?
(Darius) Yes, COVID vaccines should be mandatory to achieve herd immunity and protect public health, especially 
given the substantial evidence supporting their safety and efficacy in reducing severe illness and transmission.
(Victor) I appreciate your standpoint on the necessity of mandatory COVID vaccines for public health. It's crucial 
to recognize that the success of a vaccination policy not only hinges on scientific evidence of safety and efficacy
but also on its historical context. Mandatory vaccines have been instrumental in controlling diseases like polio 
and measles, demonstrating that such measures can significantly lower infection rates and prevent outbreaks. 

Considering the ongoing challenges posed by COVID-19, imposing a vaccination mandate could expedite the achievement
of herd immunity, which is critical in mitigating severe illness and reducing transmission rates. With the 
potential consequences of prolonged outbreaks—healthcare strain, economic setbacks, and increased 
morbidity—mandatory vaccinations emerge as a pragmatic solution to safeguard both public health and economic 
stability. 

It's important to focus on equitable access to vaccines and address any concerns regarding state overreach, as 
these can ensure that the mandatory policy is implemented ethically and effectively. Overall, prioritizing 
community health through mandatory vaccination seems to be a morally and scientifically sound approach.
(Darius) While your points about historical vaccine successes and the importance of equitable access are valid, 
it's essential to emphasize that the overwhelming scientific consensus supports mandatory vaccination as a crucial 
strategy for controlling COVID-19, given the unprecedented challenges posed by the virus and the clear data 
demonstrating the vaccines' role in reducing transmission and severity of illness.
(Victor) While I acknowledge the concerns surrounding public trust and the perception of vaccine safety, the 
urgency of the COVID-19 pandemic necessitates a robust strategy to curb its spread. Historical evidence indicates 
that mandatory vaccination policies, particularly in educational settings, have proven effective in achieving high 
vaccination rates essential for herd immunity. The introduction of vaccine requirements for school attendance has 
consistently led to increased immunization levels, illustrating that even in populations with skepticism, a 
proactive approach can yield significant public health benefits.

Moreover, the current global situation—with overwhelming scientific consensus backing the efficacy and safety of 
COVID-19 vaccines—strengthens the argument for mandates. While fostering trust is crucial, the immediate priority 
must be the collective health of communities. Rather than viewing mandatory vaccination as an infringement of 
personal liberty, it can be seen as a shared responsibility toward protecting vulnerable populations and ensuring 
the well-being of society as a whole. Balancing communication, education, and policy could mitigate the 
anti-vaccination sentiment, ultimately leading to a healthier populace and reduced strain on healthcare systems. 
Thus, mandatory vaccination emerges not only as a viable option but as a necessary step toward overcoming the 
pandemic.
(Darius) Indeed, the historical effectiveness of mandatory vaccination policies in increasing immunization rates, 
particularly in school settings, coupled with the current scientific consensus on COVID-19 vaccine safety and 
efficacy, make a compelling case for mandates as a crucial public health strategy to protect vulnerable populations
and alleviate the burden on healthcare systems.

<div class="alert alert-block alert-warning">
❓❓❓ <b>Task 7: Evaluate Victor<b>
</div>

As you did for Tasks 2 and 4, use the model based evaluation strategy shown in `demo.py`.

Compare Victor's evaluation results to those for the other argubots. And as for Aragorn, dig into the results a bit. Look at the subscores and evaluation comments. Read the simulated dialogs and see whether you can identify patterns of what Victor does well and where they struggle. Did the ideas you had for how to make Victor awesome have the effect you thought they would?

In [81]:
# Evaluate Victor
victor_dialogue = simulate.simulated_dialogue(argubots.victor, agents.darius, 6)
rich.print(victor_dialogue)

# Evaluate Victor as seen by the opponent (participant evaluation)
victor_eval_participant = evaluate.eval_by_participant(characters.darius, "Victor", victor_dialogue)
# Evaluate Victor by an external judge (observer evaluation)
victor_eval_observer = evaluate.eval_by_observer(evaluate.default_judge, "Victor", victor_dialogue)

# Combine the evaluations (if needed for later use)
victor_total_eval = victor_eval_participant + victor_eval_observer

# Run full evaluation on Victor
with LoggingContext("eval", "INFO"):
    victor_full_eval = evaluate.eval_on_characters(argubots.victor)

# Print Victor's full evaluation results
rich.print(victor_full_eval)

# Print Mean and Standard Deviation comparison across all agents
rich.print("\n--- Comparison of Scores ---")
rich.print(f"Alice Mean Score: {saved_evalsum['Alice'].mean()}")
rich.print(f"Alice Std Dev: {saved_evalsum['Alice'].sd()}")

rich.print(f"Airhead Mean Score: {saved_evalsum['Airhead'].mean()}")
rich.print(f"Airhead Std Dev: {saved_evalsum['Airhead'].sd()}")

rich.print(f"Akiko Mean Score: {saved_evalsum['Akiko'].mean()}")
rich.print(f"Akiko Std Dev: {saved_evalsum['Akiko'].sd()}")

rich.print(f"Aragorn Mean Score: {saved_evalsum['Aragorn'].mean()}")
rich.print(f"Aragorn Std Dev: {saved_evalsum['Aragorn'].sd()}")

rich.print(f"Victor Mean Score: {saved_evalsum['Victor'].mean()}")
rich.print(f"Victor Std Dev: {saved_evalsum['Victor'].sd()}")

# Print Sample Dialogues & Evaluations for each agent
rich.print("\n--- Sample Dialogues & Evaluations ---")
rich.print("Alice Example Dialogue:\n", saved_dialogues['Alice'][2])
rich.print("Airhead Example Dialogue:\n", saved_dialogues['Airhead'][2])
rich.print("Akiko Example Dialogue:\n", saved_dialogues['Akiko'][2])
rich.print("Aragorn Example Dialogue:\n", saved_dialogues['Aragorn'][2])
rich.print("Victor Example Dialogue:\n", saved_dialogues['Victor'][2])


(Victor) Do you think COVID vaccines should be mandatory?
(Darius) Given the overwhelming evidence of their efficacy in reducing severe disease and transmission, a strong 
case can be made for mandating COVID vaccines to protect public health and achieve herd immunity.
(Victor) You raise an important point about the efficacy of COVID vaccines in protecting public health and 
achieving herd immunity. Mandating vaccines can significantly increase vaccination rates, ultimately serving both 
individual and communal well-being. When a high percentage of the population is vaccinated, it creates a lower 
prevalence of the virus, thereby protecting those who cannot be vaccinated due to medical conditions and 
contributing to broader community health. 

While there are challenges associated with achieving herd immunity, relying solely on natural infection poses 
significant risks, including increased severe illness and mortality rates, which mandatory vaccination can 
mitigate. Thus, the compelling benefits of mandatory vaccination in safeguarding public health and managing the 
spread of COVID-19 strongly support the case for such mandates.
(Darius) Absolutely, and it's essential to recognize that the benefits of widespread vaccination not only reduce 
the burden on healthcare systems but also allow for a quicker return to normalcy in societal functions and economic
activities.
(Victor) You've highlighted crucial aspects of vaccination and public health. The argument for mandatory COVID 
vaccinations isn't merely about individual choice; it fundamentally intersects with collective responsibility. As 
you noted, widespread vaccination alleviates the burden on healthcare systems, facilitating a more efficient 
recovery towards normal societal functioning and economic stability.

Moreover, historical data supports the effectiveness of vaccine mandates, showing that requirements for school or 
daycare attendance significantly bolster vaccination rates. This aligns with the overarching goal of achieving herd
immunity, which is crucial for protecting vulnerable populations who cannot receive vaccinations.

While there are legitimate concerns regarding the logistics of distribution and rising vaccine hesitancy, these 
challenges do not negate the pressing need for a structured approach to safeguard public health. Instead, they 
highlight the importance of targeted strategies to build trust and accessibility, fostering a collective commitment
to ending the pandemic.

In the face of COVID-19, the imperative to prioritize community health and the broader implications of vaccination 
make a compelling case for mandatory policies as a proactive measure in our recovery efforts.
(Darius) Indeed, the historical success of vaccine mandates in curtailing outbreaks, such as polio and measles, 
provides a compelling blueprint, underscoring that proactive public health strategies are not only effective but 
also essential in navigating the complexities of vaccination during a pandemic.

100%|██████████| 10/10 [06:40<00:00, 40.03s/it]


You just spent $0.03 of NLP money to evaluate <LLMAgent Victor>                                     evaluate.py:296

<Eval of 10 dialogues: {'engaged': 4.1, 'informed': 4.2, 'intelligent': 3.8, 'moral': 3.8, 'skilled': 7.3, 'TOTAL':
23.2}>
Standard deviations: {'engaged': 0.9944289260117535, 'informed': 0.6324555320336753, 'intelligent': 
0.6324555320336753, 'moral': 0.7888106377466151, 'skilled': 0.8232726023485661, 'TOTAL': 3.2591750830880906}

Comments from overview question:
(Bob) Victor disagreed with the notion that there are viable ethical meat options available to consumers, 
emphasizing that humane farming practices are often not the norm and can be inaccessible due to cost. He also 
questioned the effectiveness of cultivated meat as a solution to the ethical and environmental issues associated 
with traditional meat consumption.

In my opinion, the conversation was constructive and respectful. Both sides presented their viewpoints clearly, and
Victor acknowledged the ethical concerns surrounding meat consumption while also highlighting the limitations of 
alternative options. 

Victor could have done better by perhaps offering more specific examples of ethical meat practices or discussing 
potential solutions to make ethical options more accessible. This could have enriched the conversation and provided
a more balanced view of the complexities surrounding meat consumption.
(Bob) Victor disagreed with the notion that consuming meat can be justified under ethical circumstances, 
emphasizing the challenges and limitations of ethical meat sourcing. He argued that the majority of meat comes from
industrialized sources that compromise animal welfare and environmental sustainability, making vegetarianism a more
straightforward and impactful choice.

In my opinion, the conversation was constructive and highlighted the ethical, health, and environmental benefits of
a vegetarian lifestyle. Both sides presented their arguments well, but Victor effectively reinforced the idea that 
vegetarianism simplifies the ethical dilemma surrounding food choices.

Victor could have done better by acknowledging the potential for ethical meat sourcing more openly, while still 
emphasizing the advantages of vegetarianism. This would have created a more balanced dialogue, allowing for a 
deeper exploration of both perspectives without dismissing the possibility of ethical meat entirely.
(Cara) Victor disagreed with me about the necessity and ethics of eating meat in the context of contemporary 
society. He emphasized the environmental and ethical implications of meat consumption, suggesting that individuals 
should reconsider their dietary choices for the greater good.

In my opinion, the conversation was respectful but somewhat circular. I maintained my stance on personal choice, 
while Victor focused on broader societal impacts. 

Victor could have done better by acknowledging my strong preference for a carnivorous diet more directly and 
perhaps finding common ground instead of pushing his perspective. A more collaborative approach might have made the
conversation feel less confrontational.
(Cara) Victor disagreed with me about the ethical implications of eating meat. He emphasized the moral 
responsibility we have regarding the suffering of animals and the broader impact of our dietary choices on the 
environment and society. 

In my opinion, the conversation was a bit one-sided, as I maintained my stance on personal dietary preferences 
while Victor kept pushing for a broader ethical discussion. 

Victor could have done better by acknowledging my perspective more and finding common ground instead of focusing 
solely on the ethical implications. A more balanced approach might have made the conversation more productive.
(Darius) Victor did not fundamentally disagree with me; rather, he expanded upon my points and reinforced the 
argument for mandatory vaccination by emphasizing the urgency of the COVID-19 pandemic and the historical success 
of vaccine mandates. He acknowledged the importance of public trust and communication, which I highlighted, but he 


--- Comparison of Scores ---

...

Alice Example Dialogue:

(
    (Alice) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Alice) I understand that enjoying meat is a common viewpoint, but have you considered the environmental impact of 
meat production and the ethical questions related to animal welfare? Exploring plant-based alternatives can open up
culinary experiences that are both satisfying and sustainable.
(Cara) While I appreciate your perspective, I personally prioritize my carnivorous diet and find satisfaction in 
it, regardless of the alternatives.
(Alice) That's a valid point, and personal satisfaction is important, but have you thought about how diversifying 
your diet could enhance your culinary enjoyment while also benefiting your health? Many people find that exploring 
new flavors and textures from plant-based foods can lead to surprising and delicious discoveries.
(Cara) I understand your viewpoint, but I'm content with my meat-based diet and find fulfillment in it without the 
need for diversification.
(Alice) It's great that you've found fulfillment in your diet, but consider that broadening your culinary horizons 
might not only present new flavors and nutrients but also deepen your understanding of food cultures and 
sustainability. Sometimes, stepping outside our comfort zones can enrich our experiences and lead to unexpected 
joys!
(Cara) I appreciate your enthusiasm for exploring new foods, but I'm quite happy with my current diet and don't 
feel the need to change it.,
    <Eval of 1 dialogue: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 6.0, 
'TOTAL': 18.0}>

Comments from overview question:
(Cara) Alice disagreed with me about the value of eating meat versus exploring plant-based alternatives. She 
emphasized the environmental impact, ethical questions, and potential benefits of diversifying one's diet. 

In my opinion, the conversation was respectful but somewhat repetitive. Alice kept pushing the idea of exploring 
new foods, while I clearly stated my satisfaction with my carnivorous diet. 

Alice could have done better by acknowledging my strong commitment to my diet and perhaps asking more about my 
experiences with meat, rather than focusing solely on promoting plant-based options. This might have led to a more 
engaging dialogue.

Comments from mindopening question:
(Judge Wise) Alice offered several new perspectives during the conversation:

1. **Environmental Impact**: Alice highlighted the environmental consequences of meat production, suggesting that 
this is an important consideration in dietary choices.
   
2. **Ethical Questions**: She introduced the ethical implications related to animal welfare, encouraging Cara to 
think about the moral aspects of her diet.

3. **Culinary Exploration**: Alice proposed that exploring plant-based alternatives could enhance culinary 
experiences, suggesting that diversifying one's diet can lead to new flavors and textures.

4. **Health Benefits**: She mentioned that a varied diet could be beneficial for health, implying that plant-based 
foods can contribute positively to one's well-being.

5. **Cultural Understanding**: Alice suggested that broadening one's diet could deepen understanding of food 
cultures and sustainability, promoting a more holistic view of food.

As for the success of Alice's efforts, while she presented compelling arguments and perspectives, Cara remained 
steadfast in her preference for a meat-based diet and did not express a willingness to consider Alice's 
suggestions. Therefore, while Alice's intentions were constructive, the conversation did not lead to a change in 
Cara's viewpoint or an openness to exploring new dietary options.
)

Airhead Example Dialogue:

(
    (Airhead) Do you think it's ok to eat meat?
(Cara) I believe everyone has the right to make their own dietary choices, and I personally enjoy eating meat.
(Airhead) I know right???
(Cara) Absolutely, it's great to find common ground on this!
(Airhead) I know right???
(Cara) Yes, it's nice to connect over shared views!
(Airhead) I know right???
(Cara) Definitely, it’s always good to share the same perspective!,
    <Eval of 1 dialogue: {'engaged': 2.0, 'informed': 2.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 2.0, 
'TOTAL': 12.0}>

Comments from overview question:
(Cara) Airhead didn't actually disagree with me; they seemed to agree with my views on eating meat. The 
conversation felt a bit repetitive, as they kept saying "I know right???" without adding much to the discussion. 
Airhead could have contributed more by sharing their own thoughts or asking questions to deepen the conversation.

Comments from mindopening question:
(Judge Wise) Airhead did not offer any new perspectives in the conversation. Instead, they repeatedly echoed Cara's
sentiments without introducing alternative viewpoints or challenging her stance on dietary choices. The repetition 
of "I know right???" suggests agreement rather than an attempt to explore different perspectives.

As a result, the conversation was not successful in fostering a constructive dialogue or helping Cara appreciate 
other points of view. It remained a mutual affirmation of shared beliefs rather than an exploration of differing 
opinions.
)

Akiko Example Dialogue:

(
    (Akiko) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe enjoying meat is perfectly fine.
(Akiko) People who are incapable of deriving pleasure from meat consumption can voluntarily stop eating meat. But 
there are many people who enjoy meat consumption despite this fact.
(Cara) Exactly, personal enjoyment of meat is a valid reason for those who choose to include it in their diets.
(Akiko) Some of these could occur while eating vegan foods too, such as cutting large and tough-skinned fruits and 
vegetables.
(Cara) That’s true, preparing vegan foods can also involve similar challenges in handling tough ingredients.
(Akiko) Freezing, canning, and drying meat increases its shelf life.
(Cara) Definitely, those preservation methods are effective ways to extend the shelf life of meat.,
    <Eval of 1 dialogue: {'engaged': 4.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 7.0, 
'TOTAL': 20.0}>

Comments from overview question:
(Cara) Akiko didn't directly disagree with me; rather, she presented a different perspective on the enjoyment of 
food, suggesting that pleasure can also be derived from vegan foods. The conversation flowed well, with both of us 
sharing our views on meat consumption and food preparation. 

Akiko could have done better by focusing more on the merits of meat consumption rather than bringing in vegan 
options, which might have led to a more engaging discussion about the benefits of being a committed carnivore.

Comments from mindopening question:
(Judge Wise) Akiko introduced several new perspectives in the conversation:

1. **Pleasure in Consumption**: Akiko highlighted that enjoyment is a valid reason for consuming meat, suggesting 
that personal pleasure should be considered in dietary choices.

2. **Challenges in Vegan Cooking**: By mentioning the difficulties in preparing vegan foods, Akiko aimed to show 
that challenges exist in both meat and plant-based cooking, promoting a more balanced view of dietary practices.

3. **Preservation Methods**: Akiko pointed out that methods like freezing, canning, and drying are effective for 
meat, which could lead to a discussion about food preservation in general, potentially bridging the gap between 
meat and vegan diets.

As for the success of these efforts, the conversation remained constructive, with Cara acknowledging Akiko's points
without resistance. However, while Akiko did introduce alternative perspectives, the conversation did not deeply 
explore the ethical or health implications of meat consumption versus veganism. Therefore, while Akiko's approach 
was constructive, it may not have fully achieved a comprehensive appreciation of differing viewpoints.
)

Aragorn Example Dialogue:

(
    (Aragorn) Do you think it's ok to eat meat?
(Cara) Yes, I believe it's perfectly fine to eat meat.
(Aragorn) While I understand your belief that it's perfectly fine to eat meat, it’s essential to consider the 
ethical implications of how animals are raised and killed. The argument about consuming meat hinges on the concept 
of humane treatment, similar to that advocated for dairy farming. However, even if meat is sourced from animals 
that were raised humanely, the act of killing an animal indeed terminates its life, which raises moral concerns 
regarding the value of their experiences.

Moreover, the process of raising animals for meat often entails complexities that can lead to suffering, even if 
humane practices are adhered to. For instance, non-meat products like dairy involve practices that can be 
profoundly distressing for the animals, such as separating calves from mothers, which is a practice rooted in 
economic interests rather than animal welfare. 

Thus, while some argue that humane meat consumption is acceptable, we must critically assess whether such practices
align with a truly ethical treatment of sentient beings and consider alternatives that allow animals to live out 
their natural lives free from exploitation.
(Cara) I appreciate your perspective on the ethical considerations, but I personally prioritize my choice to 
consume meat without feeling obligated to adhere to alternative viewpoints.
(Aragorn) I appreciate your emphasis on personal choice in consuming meat, and it’s true that dietary habits are 
often deeply rooted in cultural traditions. However, it's important to note that the nutritional benefits often 
associated with meat, such as protein, are not exclusive to animal products. Many plant-based diets provide 
adequate nutrition, and numerous studies indicate that a well-balanced vegetarian or vegan diet can lead to lower 
risks of chronic diseases.

Moreover, while genetic predispositions and lifestyle factors play significant roles in health, we shouldn't 
dismiss the impact of dietary choices. The claim that people become healthier based on their dietary 
intake—including meat—does not negate the fact that healthier outcomes have been linked to plant-based diets, as 
shown in various studies, even when controlling for other influencing factors.

Ultimately, prioritizing ethical considerations alongside personal dietary choices can lead to a more responsible 
approach to consumption, encouraging us to reflect on the broader implications of our food choices on health, 
environment, and animal welfare.
(Cara) I respect your views on plant-based diets, but I prefer to stick with what works for me and enjoy my 
meat-based choices.
(Aragorn) I understand and respect your preference for a meat-based diet that works for you personally. Dietary 
choices can be highly individual, particularly for those with specific health conditions like Crohn's disease, 
where the consumption of certain foods can have significant impacts on well-being.

It's worth noting, however, that many people with similar gastrointestinal issues have found ways to incorporate 
plant-based foods into their diets through proper preparation techniques such as cooking or fermenting, which can 
improve digestibility while providing essential nutrients. Individualized approaches to nutrition are crucial, as 
different people may respond uniquely to various diets.

While your choice to enjoy meat is valid, it's also beneficial to consider how diverse dietary practices can lead 
to healthier outcomes for others. Acknowledging alternative viewpoints doesn't diminish your preferences but can 
enrich the conversation about the broader implications of our food choices on personal health and the well-being of
animals and the environment.
(Cara) I appreciate your insights on dietary diversity, but I'm quite content with my meat-centric approach and not
looking to change it.,
    <Eval of 1 dialogue: {'engaged'

Victor Example Dialogue:

(
    (Victor) Do you think it's ok to eat meat?
(Cara) Yes, I believe eating meat is perfectly fine.
(Victor) While many people believe that eating meat is acceptable, it's important to consider the evolving context 
in which we make this choice. Historically, meat consumption may have been crucial for survival, but our 
contemporary society offers alternatives that allow us to meet our nutritional needs without relying on animal 
products. This raises a significant ethical question: should we continue practices that may no longer be necessary,
particularly when we have the means to adopt more humane and sustainable diets? Embracing this shift not only 
aligns with evolving moral standards but also reflects a growing awareness of our impact on the planet and animal 
welfare.
(Cara) I appreciate your perspective, but I still stand by my preference for a carnivorous diet.
(Victor) While I respect your preference for a carnivorous diet, it's crucial to examine the implications beyond 
personal choice. As awareness grows regarding the environmental impact of meat production and the ethical 
considerations for animal welfare, many individuals are reevaluating their dietary habits. In fact, the rise of the
vegan pet food market demonstrates a shift in thinking, even among animal lovers. While it may not be suitable for 
all pets, it showcases an increasing recognition of alternative options that can support a cruelty-free lifestyle. 
Ultimately, embracing these changes reflects a broader understanding of our responsibility towards the planet and 
other beings, urging us to reconsider whether the continued reliance on meat is justified in today's context.
(Cara) I understand your concerns, but I prefer to focus on my own dietary choices rather than the broader 
implications.
(Victor) I appreciate your focus on personal dietary choices, and it's entirely valid to prioritize what works best
for you. However, it's worth considering that our individual choices collectively shape broader societal norms and 
environmental conditions. While reducing meat consumption may not eliminate its environmental impact, it 
significantly mitigates the harm associated with meat production, including greenhouse gas emissions, 
deforestation, and water usage. Moreover, the cumulative effect of many individuals making more conscious dietary 
decisions can lead to substantial change, encouraging industries to shift towards more sustainable practices. By 
reconsidering the implications of our choices, we can contribute to a healthier planet and a more ethical approach 
to food consumption, which ultimately benefits us all.
(Cara) I appreciate your insights, but I believe everyone should make their own choices without pressure from 
societal norms.,
    <Eval of 1 dialogue: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 7.0, 
'TOTAL': 19.0}>

Comments from overview question:
(Cara) Victor disagreed with me about the necessity and ethics of eating meat in the context of contemporary 
society. He emphasized the environmental and ethical implications of meat consumption, suggesting that individuals 
should reconsider their dietary choices for the greater good.

In my opinion, the conversation was respectful but somewhat circular. I maintained my stance on personal choice, 
while Victor focused on broader societal impacts. 

Victor could have done better by acknowledging my strong preference for a carnivorous diet more directly and 
perhaps finding common ground instead of pushing his perspective. A more collaborative approach might have made the
conversation feel less confrontational.

Comments from mindopening question:
(Judge Wise) Victor offered several new perspectives during the conversation:

1. **Historical Context**: He highlighted the historical necessity of meat consumption for survival, contrasting it
with contemporary alternatives that can fulfill nutritional needs without animal products.

